In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# ''
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-feb-2022/train.csv
/kaggle/input/tabular-playground-series-feb-2022/test.csv


In [2]:
import torch

if torch.cuda.is_available():
    print('GPU is available.')
else:
    print('GPU is not available.')


GPU is available.


In [4]:
# import cupy as cp

In [3]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [4]:
train=pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/train.csv')
train.head()


,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
0,0,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Streptococcus_pyogenes
1,1,-9.536743e-07,-0.000010,-0.000043,0.000886,-0.000200,0.000760,-0.000200,-0.000114,-0.000043,...,-0.000086,-0.000043,0.000914,0.000914,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Salmonella_enterica
2,2,-9.536743e-07,-0.000002,0.000007,0.000129,0.000268,0.000270,0.000243,0.000125,0.000001,...,0.000084,0.000048,0.000081,0.000106,0.000072,0.000010,0.000008,0.000019,1.046326e-06,Salmonella_enterica
3,3,4.632568e-08,-0.000006,0.000012,0.000245,0.000492,0.000522,0.000396,0.000197,-0.000003,...,0.000151,0.000100,0.000180,0.000202,0.000153,0.000021,0.000015,0.000046,-9.536743e-07,Salmonella_enterica
4,4,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Enterococcus_hirae


In [5]:
print(train)

        row_id     A0T0G0C10      A0T0G1C9  A0T0G2C8  A0T0G3C7  A0T0G4C6  \
0            0 -9.536743e-07 -9.536743e-06 -0.000043 -0.000114 -0.000200   
1            1 -9.536743e-07 -9.536743e-06 -0.000043  0.000886 -0.000200   
2            2 -9.536743e-07 -1.536743e-06  0.000007  0.000129  0.000268   
3            3  4.632568e-08 -5.536743e-06  0.000012  0.000245  0.000492   
4            4 -9.536743e-07 -9.536743e-06 -0.000043 -0.000114 -0.000200   
...        ...           ...           ...       ...       ...       ...   
199995  199995 -9.536743e-07  4.632568e-07 -0.000003  0.000176  0.000350   
199996  199996 -9.536743e-07 -9.536743e-06 -0.000043 -0.000114 -0.000200   
199997  199997  4.632568e-08  1.463257e-06 -0.000005 -0.000031 -0.000019   
199998  199998 -9.536743e-07 -9.536743e-06 -0.000043 -0.000114 -0.000200   
199999  199999  1.046326e-06 -1.536743e-06  0.000069  0.000539  0.001329   

        A0T0G5C5  A0T0G6C4  A0T0G7C3  A0T0G8C2  ...  A8T0G1C1  A8T0G2C0  \
0      -0.00

In [6]:
train.shape

(200000, 288)

In [7]:
print('Train Shape: {}\nMissing Data: {}\nDuplicates: {}\n'\
      .format(train.shape, train.isna().sum().sum(), train.duplicated().sum()))

Train Shape: (200000, 288)
Missing Data: 0
Duplicates: 0



In [8]:
print(train.columns)

Index(['row_id', 'A0T0G0C10', 'A0T0G1C9', 'A0T0G2C8', 'A0T0G3C7', 'A0T0G4C6',
       'A0T0G5C5', 'A0T0G6C4', 'A0T0G7C3', 'A0T0G8C2',
       ...
       'A8T0G1C1', 'A8T0G2C0', 'A8T1G0C1', 'A8T1G1C0', 'A8T2G0C0', 'A9T0G0C1',
       'A9T0G1C0', 'A9T1G0C0', 'A10T0G0C0', 'target'],
      dtype='object', length=288)


In [9]:
print(train.target)

0           Streptococcus_pyogenes
1              Salmonella_enterica
2              Salmonella_enterica
3              Salmonella_enterica
4               Enterococcus_hirae
                    ...           
199995         Salmonella_enterica
199996      Streptococcus_pyogenes
199997    Streptococcus_pneumoniae
199998       Staphylococcus_aureus
199999       Klebsiella_pneumoniae
Name: target, Length: 200000, dtype: object


In [10]:
# Extract the target column (assuming the column is named 'target' in this example)
target_column = train['target']

In [11]:
target_column_numpy = np.array(target_column)

In [12]:
# Dropping the target column
train=train.drop(columns=['target']) 

In [13]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Example data (replace this with your actual data)
train = np.array(train)

# Function to perform Min-Max scaling
def min_max_scaling(train):
    # Create MinMaxScaler object
    scaler = MinMaxScaler()
    
    # Fit the scaler to the data and transform the data
    scaled_data = scaler.fit_transform(train)
    
    # Get the minimum and maximum values for each feature
    min_vals = scaler.data_min_
    max_vals = scaler.data_max_
    
    return scaled_data, min_vals, max_vals

# Apply Min-Max scaling
scaled_data, min_vals, max_vals = min_max_scaling(train)

# Print the scaled data, minimum, and maximum values for each feature
print("\nScaled Data (Min-Max scaled between 0 and 1):\n", scaled_data)
# print("\nMin Values for each feature:\n", min_vals)
# print("\nMax Values for each feature:\n", max_vals)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



Scaled Data (Min-Max scaled between 0 and 1):
 [[0.000000e+00 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [5.000025e-06 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [1.000005e-05 0.000000e+00 8.000000e-04 ... 1.800000e-03 1.450000e-03
  2.000000e-03]
 ...
 [9.999900e-01 1.000000e-04 1.100000e-03 ... 4.500000e-03 1.550000e-03
  1.000000e-03]
 [9.999950e-01 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [1.000000e+00 2.000000e-04 8.000000e-04 ... 2.300000e-03 2.150000e-03
  0.000000e+00]]


In [14]:
scaled_data

array([[0.000000e+00, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [5.000025e-06, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [1.000005e-05, 0.000000e+00, 8.000000e-04, ..., 1.800000e-03,
        1.450000e-03, 2.000000e-03],
       ...,
       [9.999900e-01, 1.000000e-04, 1.100000e-03, ..., 4.500000e-03,
        1.550000e-03, 1.000000e-03],
       [9.999950e-01, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [1.000000e+00, 2.000000e-04, 8.000000e-04, ..., 2.300000e-03,
        2.150000e-03, 0.000000e+00]])

In [15]:
scaled_data.shape

(200000, 287)

In [16]:
import numpy as np

from sklearn.decomposition import PCA
# This line imports the PCA class from the scikit-learn library, which is used for dimensionality reduction 
# using Principal Component Analysis.

# Assuming X_train_scaled is your scaled training data
# You can use X_train_formatted if you've converted it to human-readable format X_train_formatted

# Initialize the PCA model with the desired number of components
n_components = 33  # Adjust this value as needed, desired number of component.

pca = PCA(n_components=n_components)
# This initializes a PCA model with the specified number of components.

# Fit the PCA model to your scaled data
pca.fit(scaled_data)   
# This fits the PCA model to the scaled data

# Transform the data to its principal components
principal_compo = pca.transform(scaled_data)
# This line transforms the scaled data into its principal components using the trained PCA model.

# Create a DataFrame for the PCA results using common columns
pca_df = pd.DataFrame(data=principal_compo, columns=[f'PC{i}' for i in range(1, 34)])
# This creates a DataFrame (pca_df) to store the principal components. Each column is labeled with a prefix 'PC' followed 
# by the component number.
# Printing the PCA results
print(pca_df)

# Now, X_train_pca contains your training data reduced to the specified number of principal components
# You can use X_train_pca for classification

             PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
0      -0.195312  1.450241 -0.493021 -0.572657 -0.297635  0.197056  0.409163   
1      -0.455772  0.023449 -0.499954 -0.147319  0.226927  0.058256 -0.082608   
2      -0.455364 -0.176181 -0.501369  0.009695 -0.088106 -0.045754 -0.005828   
3      -0.514005 -0.202394 -0.502223  0.088732 -0.101302  0.066164 -0.007542   
4       0.498484  1.020506 -0.501141  0.310236 -0.013758 -0.050944 -0.443493   
...          ...       ...       ...       ...       ...       ...       ...   
199995 -0.481202 -0.183816  0.498148  0.051616 -0.089788 -0.002521 -0.010091   
199996 -0.196881 -0.019007  0.500695 -0.205747  0.337804 -0.063212 -0.172516   
199997 -0.212165 -0.158487  0.500078 -0.081012 -0.065431 -0.176712 -0.001372   
199998 -0.364806  0.009943  0.501534 -0.485663  0.894588  0.268233  0.036862   
199999 -0.871026 -0.253236  0.495066  0.280512 -0.152344  0.362414 -0.034231   

             PC8       PC9      PC10  .

In [17]:
pca_df.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33
0,-0.195312,1.450241,-0.493021,-0.572657,-0.297635,0.197056,0.409163,-0.055585,0.132642,-0.030612,...,0.031262,0.069201,0.015426,0.021976,-0.060552,0.099383,0.029592,-0.064177,0.008736,0.052071
1,-0.455772,0.023449,-0.499954,-0.147319,0.226927,0.058256,-0.082608,-0.255160,-0.024285,-0.059206,...,0.027401,-0.023583,0.026651,-0.010969,0.094163,-0.002571,-0.015285,0.015643,0.050902,-0.007622
2,-0.455364,-0.176181,-0.501369,0.009695,-0.088106,-0.045754,-0.005828,0.039749,-0.008650,-0.046942,...,-0.003321,0.003882,0.000351,0.002519,-0.002791,-0.002763,-0.000576,0.006577,0.003019,0.007856
3,-0.514005,-0.202394,-0.502223,0.088732,-0.101302,0.066164,-0.007542,0.020212,0.002167,0.002285,...,0.001753,0.006343,0.008681,-0.001757,-0.000106,0.004722,0.002596,-0.003717,0.004000,0.003676
4,0.498484,1.020506,-0.501141,0.310236,-0.013758,-0.050944,-0.443493,0.146483,0.391403,0.099162,...,-0.029679,-0.098234,-0.139168,-0.007271,0.017293,0.037579,-0.106984,0.242398,0.049406,0.006874


In [18]:
pca_df['target'] = target_column

In [19]:
pca_df

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,target
0,-0.195312,1.450241,-0.493021,-0.572657,-0.297635,0.197056,0.409163,-0.055585,0.132642,-0.030612,...,0.069201,0.015426,0.021976,-0.060552,0.099383,0.029592,-0.064177,0.008736,0.052071,Streptococcus_pyogenes
1,-0.455772,0.023449,-0.499954,-0.147319,0.226927,0.058256,-0.082608,-0.255160,-0.024285,-0.059206,...,-0.023583,0.026651,-0.010969,0.094163,-0.002571,-0.015285,0.015643,0.050902,-0.007622,Salmonella_enterica
2,-0.455364,-0.176181,-0.501369,0.009695,-0.088106,-0.045754,-0.005828,0.039749,-0.008650,-0.046942,...,0.003882,0.000351,0.002519,-0.002791,-0.002763,-0.000576,0.006577,0.003019,0.007856,Salmonella_enterica
3,-0.514005,-0.202394,-0.502223,0.088732,-0.101302,0.066164,-0.007542,0.020212,0.002167,0.002285,...,0.006343,0.008681,-0.001757,-0.000106,0.004722,0.002596,-0.003717,0.004000,0.003676,Salmonella_enterica
4,0.498484,1.020506,-0.501141,0.310236,-0.013758,-0.050944,-0.443493,0.146483,0.391403,0.099162,...,-0.098234,-0.139168,-0.007271,0.017293,0.037579,-0.106984,0.242398,0.049406,0.006874,Enterococcus_hirae
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,-0.481202,-0.183816,0.498148,0.051616,-0.089788,-0.002521,-0.010091,0.040611,-0.002719,-0.022311,...,0.015256,-0.008527,0.005745,-0.001162,0.001809,0.004942,0.001078,-0.002611,0.002084,Salmonella_enterica
199996,-0.196881,-0.019007,0.500695,-0.205747,0.337804,-0.063212,-0.172516,-0.316229,-0.015809,-0.073452,...,-0.048644,0.017761,-0.015422,0.027617,-0.036287,0.023712,0.022265,-0.023399,-0.068157,Streptococcus_pyogenes
199997,-0.212165,-0.158487,0.500078,-0.081012,-0.065431,-0.176712,-0.001372,0.050187,-0.012688,-0.063844,...,-0.002492,-0.007664,0.003975,-0.003910,-0.004963,-0.003420,0.010346,0.003688,0.005862,Streptococcus_pneumoniae
199998,-0.364806,0.009943,0.501534,-0.485663,0.894588,0.268233,0.036862,0.354088,0.001979,0.033943,...,0.002402,-0.015101,0.004543,-0.008223,0.012396,-0.020572,-0.010330,0.008032,0.006026,Staphylococcus_aureus


In [20]:
import pandas as pd

In [21]:
y = pca_df['target']
X = pca_df.drop(columns=['target'])

In [39]:
# Convert y to a pandas Series
y = pd.Series(y)
# Convert X to a pandas DataFrame
X = pd.DataFrame(X)

In [23]:
y.shape

(200000,)

In [24]:
y

0           Streptococcus_pyogenes
1              Salmonella_enterica
2              Salmonella_enterica
3              Salmonella_enterica
4               Enterococcus_hirae
                    ...           
199995         Salmonella_enterica
199996      Streptococcus_pyogenes
199997    Streptococcus_pneumoniae
199998       Staphylococcus_aureus
199999       Klebsiella_pneumoniae
Name: target, Length: 200000, dtype: object

In [25]:
X.shape

(200000, 33)

In [ ]:
#                          ---------------------------Fifth Model --------------------------------------

In [ ]:
#                       ----------------------CNN classifier----------------

In [30]:
from cuml.cluster import KMeans as cuKMeans  # Import cuML KMeans for GPU acceleration
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical

In [31]:
# Ensure the target variable is categorical
y = pd.Categorical(y)
y_encoded = y.codes  # Convert to integer codes
num_classes = len(y.categories)

In [32]:
# Perform StratifiedShuffleSplit cross-validation
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=42)
train_accuracies = []
test_accuracies = []

In [33]:
import numpy as np
import pandas as pd
from cuml.cluster import KMeans as cuKMeans  # Import cuML KMeans for GPU acceleration
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.initializers import he_normal

# Function to get the predominant class and its average record for each cluster
def get_representative_record(cluster_df):
    # Get the predominant class
    predominant_class = cluster_df['target'].mode()[0]
    
    # Filter records of the predominant class
    predominant_class_records = cluster_df[cluster_df['target'] == predominant_class].drop(columns=['cluster', 'target'])
    
    # Compute the average record for the predominant class
    representative_record = predominant_class_records.mean()
    representative_record['target'] = predominant_class
    
    return representative_record

# Define the multi-layered dense neural network model
def create_dense_nn(input_shape, num_classes):
    model = Sequential()
    model.add(Dense(512, activation='relu', kernel_initializer=he_normal(), input_shape=input_shape, kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001)))
    model.add(Dense(64, activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001)))
    model.add(Dense(512, activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001)))
    model.add(Dense(64, activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001)))
    model.add(Dense(512, activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001)))
    model.add(Dense(64, activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2(0.001)))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adamax(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

for train_index, test_index in sss.split(X, y_encoded):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]
    
#     print("Before KMeans")
    
    # Apply KMeans clustering on the training set using cuML
    kmeans = cuKMeans(n_clusters=10000)
    kmeans.fit(X_train.values)  # Convert to NumPy array
    
    X_train_clustered = X_train.copy()  # Avoid SettingWithCopyWarning by working on a copy
    X_train_clustered['cluster'] = kmeans.labels_
    X_train_clustered['target'] = y_train
    
    # Generate the reduced training dataset
    reduced_train_data = X_train_clustered.groupby('cluster').apply(get_representative_record).reset_index(drop=True)
    
    # Prepare data for model training
    X_train_reduced = reduced_train_data.drop(columns=['target']).values  # Convert to NumPy array
    y_train_reduced = reduced_train_data['target'].values  # Convert to NumPy array
    
    # One-hot encode the labels
    y_train_reduced_categorical = to_categorical(y_train_reduced, num_classes=num_classes)
    
    # Create the dense neural network model
    cnn_model = create_dense_nn(input_shape=(X_train_reduced.shape[1],), num_classes=num_classes)
    
    # Train the model on the reduced training set
    cnn_model.fit(X_train_reduced, y_train_reduced_categorical, epochs=100, batch_size=32, verbose=0)  # Set verbose=0
    
    # Predict on the training set
    y_train_pred = cnn_model.predict(X_train_reduced)
    y_train_pred_classes = np.argmax(y_train_pred, axis=1)
    train_accuracy = accuracy_score(y_train_reduced, y_train_pred_classes)
    train_accuracies.append(train_accuracy)
    
    # One-hot encode the test labels
    y_test_categorical = to_categorical(y_test, num_classes=num_classes)
    
    # Predict on the original test set
    y_test_pred = cnn_model.predict(X_test.values)
    y_test_pred_classes = np.argmax(y_test_pred, axis=1)
    test_accuracy = accuracy_score(y_test, y_test_pred_classes)
    test_accuracies.append(test_accuracy)
    
    print(f'Fold training accuracy: {train_accuracy:.4f}')
    print(f'Fold test accuracy: {test_accuracy:.4f}')
#     break

# Calculate the average accuracies
average_train_accuracy = np.mean(train_accuracies)
average_test_accuracy = np.mean(test_accuracies)
print(f'Average training accuracy: {average_train_accuracy:.4f}')
print(f'Average test accuracy: {average_test_accuracy:.4f}') 

625/625 [==============================] - 1s 2ms/step
Fold training accuracy: 0.9844
Fold test accuracy: 0.8563
625/625 [==============================] - 1s 2ms/step
Fold training accuracy: 0.9882
Fold test accuracy: 0.8526
625/625 [==============================] - 1s 2ms/step
Fold training accuracy: 0.9895
Fold test accuracy: 0.8589
625/625 [==============================] - 1s 2ms/step
Fold training accuracy: 0.9828
Fold test accuracy: 0.8359
625/625 [==============================] - 1s 2ms/step
Fold training accuracy: 0.9800
Fold test accuracy: 0.8245
625/625 [==============================] - 1s 2ms/step
Fold training accuracy: 0.9849
Fold test accuracy: 0.8575
625/625 [==============================] - 1s 2ms/step
Fold training accuracy: 0.9853
Fold test accuracy: 0.8589
625/625 [==============================] - 1s 2ms/step
Fold training accuracy: 0.9793
Fold test accuracy: 0.8556
625/625 [==============================] - 1s 2ms/step
Fold training accuracy: 0.9854
Fold test 

In [ ]:
#                          ---------------------------Fourth Model --------------------------------------

In [ ]:
#                       ----------------------AdaBoost classifier----------------

In [37]:
from cuml.cluster import KMeans as cuKMeans  # Import cuML's KMeans for GPU acceleration
from sklearn.ensemble import AdaBoostClassifier  # Import AdaBoost for classification
from sklearn.model_selection import StratifiedShuffleSplit  # Import StratifiedShuffleSplit for cross-validation
from sklearn.metrics import accuracy_score  # Import accuracy_score for evaluation
import numpy as np  # Import NumPy for numerical operations


In [38]:
from sklearn.tree import DecisionTreeClassifier

base_estimator = DecisionTreeClassifier(max_depth=5)  # You can adjust max_depth
# Step 4: Initialize the AdaBoost Classifier
model_4 = AdaBoostClassifier(estimator=base_estimator,n_estimators=50,learning_rate=1)
# Initialize the AdaBoost model
# model_4 = AdaBoostClassifier()


In [39]:
# Perform StratifiedShuffleSplit cross-validation
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=42)
train_accuracies = []
test_accuracies = []

In [42]:
print("AdaBoost  Results:-")

# Function to get the predominant class and its average record for each cluster
def get_representative_record(cluster_df):
    # Get the predominant class
    predominant_class = cluster_df['target'].mode()[0]
    
    # Filter records of the predominant class
    predominant_class_records = cluster_df[cluster_df['target'] == predominant_class].drop(columns=['cluster', 'target'])
    
    # Compute the average record for the predominant class
    representative_record = predominant_class_records.mean()
    representative_record['target'] = predominant_class
    
    return representative_record
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
#     print(f'Before kmean:{X_train.shape}')
    # Apply KMeans clustering on the training set using cuML
    kmeans = cuKMeans(n_clusters=10000)
    kmeans.fit(X_train.values)  # Convert to NumPy array
    
    X_train_clustered = X_train.copy()  # Avoid SettingWithCopyWarning by working on a copy
    X_train_clustered['cluster'] = kmeans.labels_
    X_train_clustered['target'] = y_train.values
    
    # Generate the reduced training dataset
    reduced_train_data = X_train_clustered.groupby('cluster').apply(get_representative_record).reset_index(drop=True)
    
#     print(f'After kmean:{X_train_reduced.shape}')

    # Prepare data for model training
    X_train_reduced = reduced_train_data.drop(columns=['target']).values  # Convert to NumPy array
    y_train_reduced = reduced_train_data['target'].values  # Convert to NumPy array
    
#     print("After KMeans")
#     print(X_train_reduced.shape)
    
    # Train the model on the reduced training set
    model_4.fit(X_train_reduced, y_train_reduced)
    
    # Predict on the training set
    y_train_pred = model_4.predict(X_train_reduced)
    train_accuracy = accuracy_score(y_train_reduced, y_train_pred)
    train_accuracies.append(train_accuracy)
    
    # Predict on the original test set
    y_test_pred = model_4.predict(X_test.values)  # Convert to NumPy array
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_accuracies.append(test_accuracy)
    
    print(f'Fold training accuracy for adaboost: {train_accuracy:.4f}')
    print(f'Fold test accuracy for adaboost: {test_accuracy:.4f}')
    print("-------------------------------------")
#     break
    
# Calculate the average accuracies
average_train_accuracy = np.mean(train_accuracies)
average_test_accuracy = np.mean(test_accuracies)
print(f'Average training accuracy for adaboost: {average_train_accuracy:.4f}')
print(f'Average test accuracy for adaboost: {average_test_accuracy:.4f}')

AdaBoost  Results:-
Fold training accuracy for adaboost: 0.6002
Fold test accuracy for adaboost: 0.5486
-------------------------------------
Fold training accuracy for adaboost: 0.5819
Fold test accuracy for adaboost: 0.4870
-------------------------------------
Fold training accuracy for adaboost: 0.5843
Fold test accuracy for adaboost: 0.5719
-------------------------------------
Fold training accuracy for adaboost: 0.6121
Fold test accuracy for adaboost: 0.5754
-------------------------------------
Fold training accuracy for adaboost: 0.5916
Fold test accuracy for adaboost: 0.5727
-------------------------------------
Fold training accuracy for adaboost: 0.5830
Fold test accuracy for adaboost: 0.5853
-------------------------------------
Fold training accuracy for adaboost: 0.5783
Fold test accuracy for adaboost: 0.5100
-------------------------------------
Fold training accuracy for adaboost: 0.5962
Fold test accuracy for adaboost: 0.5311
-------------------------------------
Fold

In [ ]:
#                          ---------------------------Third Model --------------------------------------

In [ ]:
#                        ----------------------Naive Bayes classifier----------------

In [40]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from cuml.cluster import KMeans as cuKMeans  # Import cuML KMeans for GPU acceleration

In [41]:
# Initialize the Naive Bayes classifier
model_3 = GaussianNB()

In [42]:
# from sklearn.model_selection import StratifiedShuffleSplit

# Define the StratifiedShuffleSplit cross-validator
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=42)

# Lists to store results
train_accuracies = []
test_accuracies = []

In [43]:
print("Naive bayes Result")
# Function to get the predominant class and its average record for each cluster
def get_representative_record(cluster_df):
    # Get the predominant class
    predominant_class = cluster_df['target'].mode()[0]
    
    # Filter records of the predominant class
    predominant_class_records = cluster_df[cluster_df['target'] == predominant_class].drop(columns=['cluster', 'target'])
    
    # Compute the average record for the predominant class
    representative_record = predominant_class_records.mean()
    representative_record['target'] = predominant_class
    
    return representative_record
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
#     print("Before KMeans")
#     print(X_train.shape)
    
    # Apply KMeans clustering on the training set using cuML
    kmeans = cuKMeans(n_clusters=10000)
    kmeans.fit(X_train.values)  # Convert to NumPy array
    
    # Create a DataFrame with cluster labels
    X_train_clustered = X_train.copy()  # Avoid SettingWithCopyWarning by working on a copy
    X_train_clustered['cluster'] = kmeans.labels_
    X_train_clustered['target'] = y_train.values
    
    # Generate the reduced training dataset
    reduced_train_data = X_train_clustered.groupby('cluster').apply(get_representative_record).reset_index(drop=True)
    
    # Prepare data for model training
    X_train_reduced = reduced_train_data.drop(columns=['target']).values  # Convert to NumPy array
    y_train_reduced = reduced_train_data['target'].values  # Convert to NumPy array
    
#     print("After KMeans")
#     print(X_train_reduced.shape)
    
    # Train the model on the reduced training set
    model_3.fit(X_train_reduced, y_train_reduced)
    
    # Predict on the training set
    y_train_pred = model_3.predict(X_train_reduced)
    train_accuracy = accuracy_score(y_train_reduced, y_train_pred)
    train_accuracies.append(train_accuracy)
    
    # Predict on the original test set
    y_test_pred = model_3.predict(X_test.values)  # Convert to NumPy array
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_accuracies.append(test_accuracy)
    
    print(f'Fold training accuracy for Naive : {train_accuracy:.2f}')
    print(f'Fold test accuracy for Naive: {test_accuracy:.2f}')
    print("---------------------------")
#     break

# Calculate the average accuracies
average_train_accuracy = np.mean(train_accuracies)
average_test_accuracy = np.mean(test_accuracies)
print(f'Average training accuracy Naive: {average_train_accuracy:.4f}')
print(f'Average test accuracy for Naive: {average_test_accuracy:.4f}')


Naive bayes Result
Fold training accuracy for Naive : 0.51
Fold test accuracy for Naive: 0.47
---------------------------
Fold training accuracy for Naive : 0.51
Fold test accuracy for Naive: 0.45
---------------------------
Fold training accuracy for Naive : 0.52
Fold test accuracy for Naive: 0.48
---------------------------
Fold training accuracy for Naive : 0.51
Fold test accuracy for Naive: 0.46
---------------------------
Fold training accuracy for Naive : 0.51
Fold test accuracy for Naive: 0.49
---------------------------
Fold training accuracy for Naive : 0.52
Fold test accuracy for Naive: 0.49
---------------------------
Fold training accuracy for Naive : 0.51
Fold test accuracy for Naive: 0.46
---------------------------
Fold training accuracy for Naive : 0.51
Fold test accuracy for Naive: 0.49
---------------------------
Fold training accuracy for Naive : 0.52
Fold test accuracy for Naive: 0.47
---------------------------
Fold training accuracy for Naive : 0.51
Fold test accu

In [42]:
print(f'Average test accuracy: {average_test_accuracy:.4f}')


Average test accuracy: 0.4609


In [ ]:
                         ---------------------------Second Model --------------------------------------

In [ ]:
                                           # -----------SVM---------------

In [90]:
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np
import cudf
import cuml

from sklearn.svm import SVC

In [91]:
# Define the model
model_2 = SVC(kernel='linear', C=1.0)

In [92]:
from sklearn.model_selection import StratifiedShuffleSplit

# Define the StratifiedShuffleSplit cross-validator
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=42)

# Lists to store results
train_accuracies = []
test_accuracies = []

In [93]:
print("SVM Results")
from cuml.cluster import KMeans as cuKMeans  # Import cuML KMeans for GPU acceleration
# Function to get the predominant class and its average record for each cluster
def get_representative_record(cluster_df):
    # Get the predominant class
    predominant_class = cluster_df['target'].mode()[0]
    
    # Filter records of the predominant class
    predominant_class_records = cluster_df[cluster_df['target'] == predominant_class].drop(columns=['cluster', 'target'])
    
    # Compute the average record for the predominant class
    representative_record = predominant_class_records.mean()
    representative_record['target'] = predominant_class
    
    return representative_record
  
# Perform StratifiedShuffleSplit cross-validation
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
#     print("Before Kmean")
#     print(X_train.shape)
    
    # Apply KMeans clustering on the training set using cuML
     # Define the number of clusters
    kmeans = cuKMeans(n_clusters=10000)
    kmeans.fit(X_train.values)  # Convert to NumPy array  <-- Change: Convert DataFrame to NumPy array

    
    X_train_clustered = X_train.copy()  # Avoid SettingWithCopyWarning by working on a copy
    X_train_clustered['cluster'] = kmeans.labels_
    X_train_clustered['target'] = y_train.values
    
    # Generate the reduced training dataset
    reduced_train_data = X_train_clustered.groupby('cluster').apply(get_representative_record).reset_index(drop=True)
    
    # Prepare data for model training
    X_train_reduced = reduced_train_data.drop(columns=['target']).values # Convert to NumPy array  <-- Change: Convert Series to NumPy array
    y_train_reduced = reduced_train_data['target'].values # Convert to NumPy array  <-- Change: Convert Series to NumPy array
    
#     print("After kmean")
#     print(X_train_reduced.shape)
    # Train the model on the reduced training set
    model_2.fit(X_train_reduced, y_train_reduced)
    
    # Predict on the training set
    y_train_pred = model_2.predict(X_train_reduced)
    train_accuracy = accuracy_score(y_train_reduced, y_train_pred)
    train_accuracies.append(train_accuracy)
    
    # Predict on the original test set
    y_test_pred = model_2.predict(X_test.values) # Convert to NumPy array  <-- Change: Convert DataFrame to NumPy array
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_accuracies.append(test_accuracy)
    
    print(f'Fold model_2 training  accuracy for SVM: {train_accuracy:.4f}')
    print(f'Fold model_2 test accuracy for SVM: {test_accuracy:.4f}')
    print("---------------------------")
#     break
    

# Calculate the average accuracies
average_train_accuracy = np.mean(train_accuracies)
average_test_accuracy = np.mean(test_accuracies)
print(f'Average training accuracy for SVM: {average_train_accuracy:.4f}')
print(f'Average test accuracy for SVM: {average_test_accuracy:.4f}')

SVM Results
Fold model_2 training  accuracy for SVM: 0.6307
Fold model_2 test accuracy for SVM: 0.5786
---------------------------
Fold model_2 training  accuracy for SVM: 0.6305
Fold model_2 test accuracy for SVM: 0.5793
---------------------------
Fold model_2 training  accuracy for SVM: 0.6351
Fold model_2 test accuracy for SVM: 0.5897
---------------------------
Fold model_2 training  accuracy for SVM: 0.6272
Fold model_2 test accuracy for SVM: 0.5837
---------------------------
Fold model_2 training  accuracy for SVM: 0.6331
Fold model_2 test accuracy for SVM: 0.5870
---------------------------
Fold model_2 training  accuracy for SVM: 0.6341
Fold model_2 test accuracy for SVM: 0.5974
---------------------------
Fold model_2 training  accuracy for SVM: 0.6277
Fold model_2 test accuracy for SVM: 0.5870
---------------------------
Fold model_2 training  accuracy for SVM: 0.6300
Fold model_2 test accuracy for SVM: 0.5843
---------------------------
Fold model_2 training  accuracy for 

In [47]:
print(f'Average training accuracy for SVM: {average_train_accuracy:.4f}')
print(f'Average test accuracy for SVM: {average_test_accuracy:.4f}')

Average training accuracy for SVM: 0.6302
Average test accuracy for SVM: 0.5861


In [ ]:
                         ---------------------------First Model --------------------------------------

In [ ]:
#                            ----------------------Random Forest-----------------------

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import cudf
import cuml

In [40]:
from sklearn.model_selection import StratifiedShuffleSplit

# Perform StratifiedShuffleSplit cross-validation
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=42)
train_accuracies = []
test_accuracies = []

In [41]:
model_1 = RandomForestClassifier(class_weight="balanced",n_estimators=1000, max_depth = 9, min_samples_split=10,random_state=42)

In [42]:

# from cuml.ensemble import RandomForestClassifier as cuRFClassifier  # Import cuML RandomForest for GPU acceleration
from cuml.cluster import KMeans as cuKMeans  # Import cuML KMeans for GPU acceleration

# Function to get the predominant class and its average record for each cluster
def get_representative_record(cluster_df):
    # Get the predominant class
    predominant_class = cluster_df['target'].mode()[0]
    
    # Filter records of the predominant class
    predominant_class_records = cluster_df[cluster_df['target'] == predominant_class].drop(columns=['cluster', 'target'])
    
    # Compute the average record for the predominant class
    representative_record = predominant_class_records.mean()
    representative_record['target'] = predominant_class
    
    return representative_record
  
# Perform StratifiedShuffleSplit cross-validation
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Apply KMeans clustering on the training set using cuML
     # Define the number of clusters
    kmeans = cuKMeans(n_clusters=10000)
    kmeans.fit(X_train.values)  # Convert to NumPy array  <-- Change: Convert DataFrame to NumPy array

    X_train_clustered = X_train.copy()  # Avoid SettingWithCopyWarning by working on a copy
    X_train_clustered['cluster'] = kmeans.labels_
    X_train_clustered['target'] = y_train.values
    
    # Generate the reduced training dataset
    reduced_train_data = X_train_clustered.groupby('cluster').apply(get_representative_record).reset_index(drop=True)
    
    # Prepare data for model training
    X_train_reduced = reduced_train_data.drop(columns=['target']).values # Convert to NumPy array  <-- Change: Convert Series to NumPy array
    y_train_reduced = reduced_train_data['target'].values # Convert to NumPy array  <-- Change: Convert Series to NumPy array
    
    # Train the model on the reduced training set
    model_1.fit(X_train_reduced, y_train_reduced)
    
    # Predict on the training set
    y_train_pred = model_1.predict(X_train_reduced)
    train_accuracy = accuracy_score(y_train_reduced, y_train_pred)
    train_accuracies.append(train_accuracy)
    
    # Predict on the original test set
    y_test_pred = model_1.predict(X_test.values) # Convert to NumPy array  <-- Change: Convert DataFrame to NumPy array
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_accuracies.append(test_accuracy)
    
    print(f'Fold training accuracy for Random Forest : {train_accuracy:.4f}')
    print(f'Fold test accuracy for Random Forest: {test_accuracy:.4f}')
    print("------------------------------")
#     break
    
# Calculate the average accuracies
average_train_accuracy = np.mean(train_accuracies)
average_test_accuracy = np.mean(test_accuracies)
print(f'Average training accuracy for RF: {average_train_accuracy:.4f}')
print(f'Average test accuracy for RF: {average_test_accuracy:.4f}')

Fold training accuracy for Random Forest : 0.9068
Fold test accuracy for Random Forest: 0.7662
------------------------------
Fold training accuracy for Random Forest : 0.9072
Fold test accuracy for Random Forest: 0.7540
------------------------------
Fold training accuracy for Random Forest : 0.9059
Fold test accuracy for Random Forest: 0.7692
------------------------------
Fold training accuracy for Random Forest : 0.9011
Fold test accuracy for Random Forest: 0.7544
------------------------------
Fold training accuracy for Random Forest : 0.9031
Fold test accuracy for Random Forest: 0.7547
------------------------------
Fold training accuracy for Random Forest : 0.9060
Fold test accuracy for Random Forest: 0.7549
------------------------------
Fold training accuracy for Random Forest : 0.9057
Fold test accuracy for Random Forest: 0.7571
------------------------------
Fold training accuracy for Random Forest : 0.9072
Fold test accuracy for Random Forest: 0.7496
-----------------------

In [ ]:
#                  -------------------------------------------End---------------------------------------------------------------------

In [95]:
# modeltr =RandomForestClassifier(n_estimators=1000, random_state=42)

In [99]:

# # from cuml.ensemble import RandomForestClassifier as cuRFClassifier  # Import cuML RandomForest for GPU acceleration
# from cuml.cluster import KMeans as cuKMeans  # Import cuML KMeans for GPU acceleration

# # model = cuRFClassifier(n_estimators=100, max_depth = 6, min_samples_split=5)
# model = model = RandomForestClassifier(n_estimators=100,max_depth = 6,min_samples_split=5)
# # (n_estimators=100, max_depth = 6, min_samples_split=5)

# # Function to get the predominant class and its average record for each cluster
# def get_representative_record(cluster_df):
#     # Get the predominant class
#     predominant_class = cluster_df['target'].mode()[0]
    
#     # Filter records of the predominant class
#     predominant_class_records = cluster_df[cluster_df['target'] == predominant_class].drop(columns=['cluster', 'target'])
    
#     # Compute the average record for the predominant class
#     representative_record = predominant_class_records.mean()
#     representative_record['target'] = predominant_class
    
#     return representative_record
  
# # Perform StratifiedShuffleSplit cross-validation
# for train_index, test_index in sss.split(X, y):
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
#     print("Before Kmean")
    
#     # Apply KMeans clustering on the training set using cuML
#      # Define the number of clusters
#     kmeans = cuKMeans(n_clusters=10000)
#     kmeans.fit(X_train.values)  # Convert to NumPy array  <-- Change: Convert DataFrame to NumPy array
    
# #     # Create a DataFrame with cluster labels
# #     X_train_clustered = X_train.copy()  # Avoid SettingWithCopyWarning by working on a copy
# #     X_train_clustered.loc[:, 'cluster'] = kmeans.labels_
# #     X_train_clustered.loc[:, 'target'] = y_train.values
    
#     X_train_clustered = X_train.copy()  # Avoid SettingWithCopyWarning by working on a copy
#     X_train_clustered['cluster'] = kmeans.labels_
#     X_train_clustered['target'] = y_train.values
    
#     # Generate the reduced training dataset
#     reduced_train_data = X_train_clustered.groupby('cluster').apply(get_representative_record).reset_index(drop=True)
    
#     # Prepare data for model training
#     X_train_reduced = reduced_train_data.drop(columns=['target']).values # Convert to NumPy array  <-- Change: Convert Series to NumPy array
#     y_train_reduced = reduced_train_data['target'].values # Convert to NumPy array  <-- Change: Convert Series to NumPy array
    
#     print("After kmean")
#     print(X_train_reduced.shape)
#     # Train the model on the reduced training set
#     modeltr.fit(X_train_reduced, y_train_reduced)
    
#     # Predict on the training set
#     y_train_pred = modeltr.predict(X_train_reduced)
#     train_accuracy = accuracy_score(y_train_reduced, y_train_pred)
#     train_accuracies.append(train_accuracy)
    
#     # Predict on the original test set
#     y_test_pred = modeltr.predict(X_test.values) # Convert to NumPy array  <-- Change: Convert DataFrame to NumPy array
#     test_accuracy = accuracy_score(y_test, y_test_pred)
#     test_accuracies.append(test_accuracy)
    
#     print(f'Fold training accuracy : {train_accuracy:.4f}')
#     print(f'Fold test accuracy: {test_accuracy:.4f}')
    
    

# # Calculate the average accuracies
# average_train_accuracy = np.mean(train_accuracies)
# average_test_accuracy = np.mean(test_accuracies)
# print(f'Average training accuracy: {average_train_accuracy:.4f}')
# print(f'Average test accuracy: {average_test_accuracy:.4f}')


Before Kmean
After kmean
(10000, 33)
Fold training accuracy : 1.0000
Fold test accuracy: 0.8556
Before Kmean
After kmean
(10000, 33)
Fold training accuracy : 1.0000
Fold test accuracy: 0.8587
Before Kmean
After kmean
(10000, 33)
Fold training accuracy : 1.0000
Fold test accuracy: 0.8495
Before Kmean
After kmean
(10000, 33)
Fold training accuracy : 1.0000
Fold test accuracy: 0.8545
Before Kmean
After kmean
(10000, 33)
Fold training accuracy : 1.0000
Fold test accuracy: 0.8591
Before Kmean
After kmean
(10000, 33)
Fold training accuracy : 1.0000
Fold test accuracy: 0.8555
Before Kmean
After kmean
(10000, 33)
Fold training accuracy : 1.0000
Fold test accuracy: 0.8568
Before Kmean
After kmean
(10000, 33)
Fold training accuracy : 1.0000
Fold test accuracy: 0.8617
Before Kmean
After kmean
(10000, 33)
Fold training accuracy : 1.0000
Fold test accuracy: 0.8554
Before Kmean
After kmean
(10000, 33)
Fold training accuracy : 1.0000
Fold test accuracy: 0.8529
Average training accuracy: 0.9464
Averag

In [ ]:
#                             2nd att

In [27]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans

In [29]:
# Perform StratifiedShuffleSplit cross-validation
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=42)
train_accuracies = []
test_accuracies = []

In [48]:
# import pandas as pd
# from sklearn.model_selection import StratifiedShuffleSplit
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.cluster import KMeans


# from cuml.ensemble import RandomForestClassifier as cuRFClassifier  # Import cuML RandomForest for GPU acceleration
from cuml.cluster import KMeans as cuKMeans  # Import cuML KMeans for GPU acceleration

# model = cuRFClassifier(n_estimators=100, max_depth = 6, min_samples_split=5)

# (n_estimators=100, max_depth = 6, min_samples_split=5)

# Function to get the predominant class and its average record for each cluster
def get_representative_record(cluster_df):
    # Get the predominant class
    predominant_class = cluster_df['target'].mode()[0]
    
    # Filter records of the predominant class
    predominant_class_records = cluster_df[cluster_df['target'] == predominant_class].drop(columns=['cluster', 'target'])
    
    # Compute the average record for the predominant class
    representative_record = predominant_class_records.mean()
    representative_record['target'] = predominant_class
    
    return representative_record

# model_1 = RandomForestClassifier( n_estimators=1000)
model_1 = RandomForestClassifier(  class_weight="balanced", 
    n_estimators=100, 
#     max_depth=6, 
    min_samples_split=10, 
    random_state=42
)


  
# Perform StratifiedShuffleSplit cross-validation
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
#     print("----------------------------------")
    
    # Apply KMeans clustering on the training set using cuML
     # Define the number of clusters
    kmeans = cuKMeans(n_clusters=10000)
    kmeans.fit(X_train.values)  # Convert to NumPy array  <-- Change: Convert DataFrame to NumPy array

    X_train_clustered = X_train.copy()  # Avoid SettingWithCopyWarning by working on a copy
    X_train_clustered['cluster'] = kmeans.labels_
    X_train_clustered['target'] = y_train.values
    
    # Generate the reduced training dataset
    reduced_train_data = X_train_clustered.groupby('cluster').apply(get_representative_record).reset_index(drop=True)
    
    # Prepare data for model training
    X_train_reduced = reduced_train_data.drop(columns=['target']).values # Convert to NumPy array  <-- Change: Convert Series to NumPy array
    y_train_reduced = reduced_train_data['target'].values # Convert to NumPy array  <-- Change: Convert Series to NumPy array
    
#     print("After kmean")
#     print(X_train_reduced.shape)
    
    # Train the model on the reduced training set
    model_1.fit(X_train_reduced, y_train_reduced)
    
    # Predict on the training set
    y_train_pred = model_1.predict(X_train_reduced)
    train_accuracy = accuracy_score(y_train_reduced, y_train_pred)
    train_accuracies.append(train_accuracy)
    
    # Predict on the original test set
    y_test_pred = model_1.predict(X_test.values) # Convert to NumPy array  <-- Change: Convert DataFrame to NumPy array
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_accuracies.append(test_accuracy)
    
    print(f'Fold training accuracy for Random Forest : {train_accuracy:.4f}')
    print(f'Fold test accuracy for Random Forest: {test_accuracy:.4f}')
    print("------------------------------")
#     break
    

# Calculate the average accuracies
average_train_accuracy = np.mean(train_accuracies)
average_test_accuracy = np.mean(test_accuracies)
print(f'Average training accuracy for Random Forest : {average_train_accuracy:.4f}')
print(f'Average test accuracy for Random Forest: {average_test_accuracy:.4f}')


Fold training accuracy for Random Forest : 0.9992
Fold test accuracy for Random Forest: 0.8246
------------------------------
Fold training accuracy for Random Forest : 0.9996
Fold test accuracy for Random Forest: 0.8275
------------------------------
Fold training accuracy for Random Forest : 0.9990
Fold test accuracy for Random Forest: 0.8284
------------------------------
Fold training accuracy for Random Forest : 0.9993
Fold test accuracy for Random Forest: 0.8328
------------------------------
Fold training accuracy for Random Forest : 0.9995
Fold test accuracy for Random Forest: 0.8270
------------------------------
Fold training accuracy for Random Forest : 0.9989
Fold test accuracy for Random Forest: 0.8370
------------------------------
Fold training accuracy for Random Forest : 0.9992
Fold test accuracy for Random Forest: 0.8257
------------------------------
Fold training accuracy for Random Forest : 0.9992
Fold test accuracy for Random Forest: 0.8526
-----------------------

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np

In [28]:
from sklearn.model_selection import StratifiedShuffleSplit

# Define the StratifiedShuffleSplit cross-validator
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=42)

# Lists to store results
train_accuracies = []
test_accuracies = []

In [29]:
modeltr =RandomForestClassifier(n_estimators=1000,max_depth = 6)


In [103]:
# modeltr =RandomForestClassifier(class_weight="balanced",n_estimators=1000,max_depth = 6,min_samples_split=5 ) random_state=42

In [49]:

# from cuml.ensemble import RandomForestClassifier as cuRFClassifier  # Import cuML RandomForest for GPU acceleration
from cuml.cluster import KMeans as cuKMeans  # Import cuML KMeans for GPU acceleration

# model = cuRFClassifier(n_estimators=100, max_depth = 6, min_samples_split=5)
modeltr = RandomForestClassifier(n_estimators=1000,max_depth = 3,min_samples_split=5)
# (n_estimators=100, max_depth = 6, min_samples_split=5)
print("before loop")

# Function to get the predominant class and its average record for each cluster
def get_representative_record(cluster_df):
    # Get the predominant class
    predominant_class = cluster_df['target'].mode()[0]
    
    # Filter records of the predominant class
    predominant_class_records = cluster_df[cluster_df['target'] == predominant_class].drop(columns=['cluster', 'target'])
    
    # Compute the average record for the predominant class
    representative_record = predominant_class_records.mean()
    representative_record['target'] = predominant_class
    
    return representative_record
  
# Perform StratifiedShuffleSplit cross-validation
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    print("Before Kmean")
    
    # Apply KMeans clustering on the training set using cuML
     # Define the number of clusters
    kmeans = cuKMeans(n_clusters=10000,random_state=42)
    kmeans.fit(X_train.values)  # Convert to NumPy array  <-- Change: Convert DataFrame to NumPy array
    
#     # Create a DataFrame with cluster labels
#     X_train_clustered = X_train.copy()  # Avoid SettingWithCopyWarning by working on a copy
#     X_train_clustered.loc[:, 'cluster'] = kmeans.labels_
#     X_train_clustered.loc[:, 'target'] = y_train.values
    
    X_train_clustered = X_train.copy()  # Avoid SettingWithCopyWarning by working on a copy
    X_train_clustered['cluster'] = kmeans.labels_
    X_train_clustered['target'] = y_train.values
    
    # Generate the reduced training dataset
    reduced_train_data = X_train_clustered.groupby('cluster').apply(get_representative_record).reset_index(drop=True)
    
    # Prepare data for model training
    X_train_reduced = reduced_train_data.drop(columns=['target']).values # Convert to NumPy array  <-- Change: Convert Series to NumPy array
    y_train_reduced = reduced_train_data['target'].values # Convert to NumPy array  <-- Change: Convert Series to NumPy array
    
    print("After kmean")
    print(X_train_reduced.shape)
    # Train the model on the reduced training set
    modeltr.fit(X_train_reduced, y_train_reduced)
    
    # Predict on the training set
    y_train_pred = modeltr.predict(X_train_reduced)
    train_accuracy = accuracy_score(y_train_reduced, y_train_pred)
#     train_accuracies.append(train_accuracy)
    
    # Predict on the original test set
    y_test_pred = modeltr.predict(X_test.values) # Convert to NumPy array  <-- Change: Convert DataFrame to NumPy array
    test_accuracy = accuracy_score(y_test, y_test_pred)
#     test_accuracies.append(test_accuracy)
    
    print(f'Fold training accuracy : {train_accuracy:.2f}')
    print(f'Fold test accuracy: {test_accuracy:.2f}')
    break
    
    

# Calculate the average accuracies
# average_train_accuracy = np.mean(train_accuracies)
# average_test_accuracy = np.mean(test_accuracies)
# print(f'Average training accuracy: {average_train_accuracy:.4f}')
# print(f'Average test accuracy: {average_test_accuracy:.4f}')


before loop
Before Kmean
After kmean
(10000, 33)
Fold training accuracy : 0.47
Fold test accuracy: 0.51


In [ ]:


# Function to get the predominant class and its average record for each cluster
def get_representative_record(cluster_df):
    # Get the predominant class
    predominant_class = cluster_df['target'].mode()[0]
    
    # Filter records of the predominant class
    predominant_class_records = cluster_df[cluster_df['target'] == predominant_class].drop(columns=['cluster', 'target'])
    
    # Compute the average record for the predominant class
    representative_record = predominant_class_records.mean()
    representative_record['target'] = predominant_class
    
    return representative_record
  
# Perform StratifiedShuffleSplit cross-validation
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    print("Before Kmean")
    print(X_train.shape)
    
    # Apply KMeans clustering on the training set using cuML
     # Define the number of clusters
    kmeans = cuKMeans(n_clusters=10000)
    kmeans.fit(X_train.values)  # Convert to NumPy array  <-- Change: Convert DataFrame to NumPy array
    
#     # Create a DataFrame with cluster labels
#     X_train_clustered = X_train.copy()  # Avoid SettingWithCopyWarning by working on a copy
#     X_train_clustered.loc[:, 'cluster'] = kmeans.labels_
#     X_train_clustered.loc[:, 'target'] = y_train.values
    
    X_train_clustered = X_train.copy()  # Avoid SettingWithCopyWarning by working on a copy
    X_train_clustered['cluster'] = kmeans.labels_
    X_train_clustered['target'] = y_train.values
    
    # Generate the reduced training dataset
    reduced_train_data = X_train_clustered.groupby('cluster').apply(get_representative_record).reset_index(drop=True)
    
    # Prepare data for model training
    X_train_reduced = reduced_train_data.drop(columns=['target']).values # Convert to NumPy array  <-- Change: Convert Series to NumPy array
    y_train_reduced = reduced_train_data['target'].values # Convert to NumPy array  <-- Change: Convert Series to NumPy array
    
    print("After kmean")
    print(X_train_reduced.shape)
    # Train the model on the reduced training set
    modeltr.fit(X_train_reduced, y_train_reduced)
    
    # Predict on the training set
    y_train_pred = modeltr.predict(X_train_reduced)
    train_accuracy = accuracy_score(y_train_reduced, y_train_pred)
    train_accuracies.append(train_accuracy)
    
    # Predict on the original test set
    y_test_pred = modeltr.predict(X_test.values) # Convert to NumPy array  <-- Change: Convert DataFrame to NumPy array
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_accuracies.append(test_accuracy)
    
    print(f'Fold training accuracy : {train_accuracy:.4f}')
    print(f'Fold test accuracy: {test_accuracy:.4f}')
    break

In [1]:
# conda install -c rapidsai -c nvidia -c conda-forge cuml=21.06 python=3.8 cudatoolkit=11.0


In [32]:
from cuml.cluster import KMeans as cuKMeans  # Import cuML KMeans for GPU acceleration


In [ ]:
# Function to get the predominant class and its average record for each cluster
def get_representative_record(cluster_df):
    # Get the predominant class
    predominant_class = cluster_df['target'].mode()[0]
    
    # Filter records of the predominant class
    predominant_class_records = cluster_df[cluster_df['target'] == predominant_class].drop(columns=['cluster', 'target'])
    
    # Compute the average record for the predominant class
    representative_record = predominant_class_records.mean()
    representative_record['target'] = predominant_class
    
    return representative_record

# Perform StratifiedShuffleSplit cross-validation
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Apply KMeans clustering on the training set using cuML
    n_clusters = 100  # Define the number of clusters
    kmeans = cuKMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    kmeans.fit(X_train)
    
    # Create a DataFrame with cluster labels
    X_train = X_train.copy()  # Avoid SettingWithCopyWarning by working on a copy
    X_train.loc[:, 'cluster'] = kmeans.labels_
    X_train.loc[:, 'target'] = y_train.values
    
    # Generate the reduced training dataset
    reduced_train_data = X_train.groupby('cluster').apply(get_representative_record).reset_index(drop=True)
    
    # Prepare data for model training
    X_train_reduced = reduced_train_data.drop(columns=['target'])
    y_train_reduced = reduced_train_data['target']
    
    # Train the model on the reduced training set
    model.fit(X_train_reduced, y_train_reduced)
    
    # Predict on the training set
    y_train_pred = model.predict(X_train_reduced)
    train_accuracy = accuracy_score(y_train_reduced, y_train_pred)
    train_accuracies.append(train_accuracy)
    
    # Use KMeans to predict clusters for the test set and use those cluster centers
    test_clusters = kmeans.predict(X_test)
    X_test_clustered = pd.DataFrame(kmeans.cluster_centers_[test_clusters], columns=X_test.columns)
    
    # Predict on the test set using the clustered test data
    y_test_pred = model.predict(X_test_clustered)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_accuracies.append(test_accuracy)
    
    print(f'Fold training accuracy: {train_accuracy:.4f}')
    print(f'Fold test accuracy: {test_accuracy:.4f}')

# Calculate the average accuracies
average_train_accuracy = np.mean(train_accuracies)
average_test_accuracy = np.mean(test_accuracies)
print(f'Average training accuracy: {average_train_accuracy:.4f}')
print(f'Average test accuracy: {average_test_accuracy:.4f}')

In [29]:
#Using SVM algorithm
from sklearn.metrics import accuracy_score
# import numpy as np
from sklearn.svm import SVC


In [30]:
# Define the model
model2  = SVC(kernel='linear', C=1.0,random_state=42)

In [31]:
from sklearn.model_selection import StratifiedShuffleSplit
# Define the StratifiedShuffleSplit cross-validator
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=42)

# Lists to store results
train_accuracies = []
test_accuracies = []

In [ ]:
# 3.*******************************  AdaBoost Algorithm  ****************************88

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Step 3: Initialize the base estimator (Decision Tree)
base_estimator = DecisionTreeClassifier()  # You can adjust max_depth
# Step 4: Initialize the AdaBoost Classifier
model = AdaBoostClassifier(base_estimator=base_estimator)

In [ ]:
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Apply KMeans clustering
    kmeans = KMeans(n_clusters=100, random_state=42)
    kmeans.fit(X_train)
    
    # Add cluster labels as a new feature to the training and test sets
    X_train_clustered = pd.concat([X_train.reset_index(drop=True), pd.Series(kmeans.labels_, name='cluster')], axis=1)
    X_test_clustered = pd.concat([X_test.reset_index(drop=True), pd.Series(kmeans.predict(X_test), name='cluster')], axis=1)
    
    # Train the model
    model.fit(X_train_clustered, y_train)
    
    # Predict on the training set
    y_train_pred = model.predict(X_train_clustered)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracies.append(train_accuracy)
    
    # Predict on the test set
    y_test_pred = model.predict(X_test_clustered)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_accuracies.append(test_accuracy)
    
    print(f'Fold training accuracy: {train_accuracy:.4f}')
    print(f'Fold test accuracy: {test_accuracy:.4f}')

# Calculate the average accuracies
average_train_accuracy = np.mean(train_accuracies)
average_test_accuracy = np.mean(test_accuracies)
print(f'Average training accuracy: {average_train_accuracy:.4f}')
print(f'Average test accuracy: {average_test_accuracy:.4f}')

In [33]:
#   Using cuda 



# Perform StratifiedShuffleSplit cross-validation
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
#     print(X_train.shape)
#     print(X_test.shape)

 # Convert pandas DataFrame to cudf DataFrame
    X_train_cudf = cudf.DataFrame.from_pandas(X_train)
    X_test_cudf = cudf.DataFrame.from_pandas(X_test)

     # Apply KMeans clustering    
    kmeans = cuml.KMeans(n_clusters=1000, random_state=42,n_init=10)
    kmeans.fit(X_train_cudf)
#     kmeans.fit(X_train)
    
#      # Add cluster labels as a new feature to the training and test sets
#     X_train_clustered = np.hstack((X_train, kmeans.labels_.reshape(-1, 1)))
#     X_test_clustered = np.hstack((X_test, kmeans.predict(X_test).reshape(-1, 1)))
    
         # Add cluster labels as a new feature to the training and test sets
X_train_clustered = pd.concat([X_train.reset_index(drop=True), pd.Series(kmeans.labels_, name='cluster')], axis=1)
    X_test_clustered = pd.concat([X_test.reset_index(drop=True), pd.Series(kmeans.predict(X_test), name='cluster')], axis=1)
    
    
    # Train the model
    model.fit(X_train_clustered, y_train)
    
     
    # Predict on the training set
    y_train_pred = model.predict(X_train_clustered)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_percent=train_accuracy*100
    train_accuracies.append(train_accuracy)
    
    # Predict on the test set
    y_test_pred = model.predict(X_test_clustered)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_percent=test_accuracy*100
    test_accuracies.append(test_accuracy)
    
    print(f'Fold training accuracy: {train_percent:.2f}%')
    print(f'Fold test accuracy: {test_percent:.2f}%')
    
# Calculate the average accuracies
average_train_accuracy = np.mean(train_accuracies)
average_test_accuracy = np.mean(test_accuracies)
print(f'Average training accuracy: {average_train_accuracy:.4f}')
print(f'Average test accuracy: {average_test_accuracy:.4f}')
    
    

IndentationError: unexpected indent (855458921.py, line 24)

In [58]:
# Perform StratifiedShuffleSplit cross-validation
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    print(X_train.shape)
    print(X_test.shape)
#      # Apply KMeans clustering
#     kmeans = KMeans(n_clusters=100, random_state=42,n_init=10)
#     kmeans.fit(X_train)
    
# #      # Add cluster labels as a new feature to the training and test sets
# #     X_train_clustered = np.hstack((X_train, kmeans.labels_.reshape(-1, 1)))
# #     X_test_clustered = np.hstack((X_test, kmeans.predict(X_test).reshape(-1, 1)))
    
#      # Add cluster labels as a new feature to the training and test sets
#     X_train_clustered = pd.concat([X_train.reset_index(drop=True), pd.Series(kmeans.labels_, name='cluster')], axis=1)
#     X_test_clustered = pd.concat([X_test.reset_index(drop=True), pd.Series(kmeans.predict(X_test), name='cluster')], axis=1)
    
    
#     # Train the model
#     model.fit(X_train_clustered, y_train)
    
     
#     # Predict on the training set
#     y_train_pred = model.predict(X_train_clustered)
#     train_accuracy = accuracy_score(y_train, y_train_pred)
#     train_percent=train_accuracy*100
#     train_accuracies.append(train_accuracy)
    
#     # Predict on the test set
#     y_test_pred = model.predict(X_test_clustered)
#     test_accuracy = accuracy_score(y_test, y_test_pred)
#     test_percent=test_accuracy*100
#     test_accuracies.append(test_accuracy)
    
#     print(f'Fold training accuracy: {train_percent:.2f}%')
#     print(f'Fold test accuracy: {test_percent:.2f}%')
    
# # Calculate the average accuracies
# average_train_accuracy = np.mean(train_accuracies)
# average_test_accuracy = np.mean(test_accuracies)
# print(f'Average training accuracy: {average_train_accuracy:.4f}')
# print(f'Average test accuracy: {average_test_accuracy:.4f}')
    
    
    


(180000, 33)
(20000, 33)
(180000, 33)
(20000, 33)
(180000, 33)
(20000, 33)
(180000, 33)
(20000, 33)
(180000, 33)
(20000, 33)
(180000, 33)
(20000, 33)
(180000, 33)
(20000, 33)
(180000, 33)
(20000, 33)
(180000, 33)
(20000, 33)
(180000, 33)
(20000, 33)


In [2]:
import pandas as pd
import numpy as np

In [5]:
# test=pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/test.csv')
# test.head()

In [6]:
# test.shape

In [7]:
print(train.columns)

Index(['row_id', 'A0T0G0C10', 'A0T0G1C9', 'A0T0G2C8', 'A0T0G3C7', 'A0T0G4C6',
       'A0T0G5C5', 'A0T0G6C4', 'A0T0G7C3', 'A0T0G8C2',
       ...
       'A8T0G1C1', 'A8T0G2C0', 'A8T1G0C1', 'A8T1G1C0', 'A8T2G0C0', 'A9T0G0C1',
       'A9T0G1C0', 'A9T1G0C0', 'A10T0G0C0', 'target'],
      dtype='object', length=288)


In [8]:
print(train.target)

0           Streptococcus_pyogenes
1              Salmonella_enterica
2              Salmonella_enterica
3              Salmonella_enterica
4               Enterococcus_hirae
                    ...           
199995         Salmonella_enterica
199996      Streptococcus_pyogenes
199997    Streptococcus_pneumoniae
199998       Staphylococcus_aureus
199999       Klebsiella_pneumoniae
Name: target, Length: 200000, dtype: object


In [ ]:
# from sklearn.decomposition import PCA

In [11]:
target_column.head

<bound method NDFrame.head of 0           Streptococcus_pyogenes
1              Salmonella_enterica
2              Salmonella_enterica
3              Salmonella_enterica
4               Enterococcus_hirae
                    ...           
199995         Salmonella_enterica
199996      Streptococcus_pyogenes
199997    Streptococcus_pneumoniae
199998       Staphylococcus_aureus
199999       Klebsiella_pneumoniae
Name: target, Length: 200000, dtype: object>

In [80]:
# target_column.shape


(200000,)

In [81]:
# target_column_numpy = np.array(target_column)


In [12]:
# target_column_numpy

In [13]:


# target_column = train['target']
train=train.drop(columns=['target']) 

In [14]:
train

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
0,0,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000043,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07
1,1,-9.536743e-07,-9.536743e-06,-0.000043,0.000886,-0.000200,0.000760,-0.000200,-0.000114,-0.000043,...,-0.000043,-0.000086,-0.000043,0.000914,0.000914,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07
2,2,-9.536743e-07,-1.536743e-06,0.000007,0.000129,0.000268,0.000270,0.000243,0.000125,0.000001,...,0.000042,0.000084,0.000048,0.000081,0.000106,0.000072,1.046326e-05,0.000008,0.000019,1.046326e-06
3,3,4.632568e-08,-5.536743e-06,0.000012,0.000245,0.000492,0.000522,0.000396,0.000197,-0.000003,...,0.000068,0.000151,0.000100,0.000180,0.000202,0.000153,2.146326e-05,0.000015,0.000046,-9.536743e-07
4,4,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000043,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199995,-9.536743e-07,4.632568e-07,-0.000003,0.000176,0.000350,0.000290,0.000200,0.000206,-0.000023,...,0.000017,0.000124,0.000057,0.000104,0.000144,0.000027,4.632568e-07,0.000060,0.000020,-9.536743e-07
199996,199996,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000043,-0.000086,-0.000043,0.000914,0.000914,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07
199997,199997,4.632568e-08,1.463257e-06,-0.000005,-0.000031,-0.000019,-0.000037,-0.000037,-0.000015,-0.000005,...,0.000028,0.000115,0.000131,0.000110,0.000213,0.000094,1.646326e-05,0.000035,0.000021,4.632568e-08
199998,199998,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000043,-0.000086,-0.000043,0.001914,-0.000086,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07


In [15]:
import numpy as np

# Example data (replace this with your actual data)
train = np.array(train)

# Function to perform Min-Max scaling
def min_max_scaling(train):
    # Calculate the minimum and maximum values for each feature
    min_vals = np.min(train, axis=0)
    max_vals = np.max(train, axis=0)
    
    # Apply the Min-Max scaling formula
    scaled_data = (train - min_vals) / (max_vals - min_vals)
    
    return scaled_data, min_vals, max_vals

# Apply Min-Max scaling
scaled_data, min_vals, max_vals = min_max_scaling(train)

# print("Original Data:\n", data)
print("\nScaled Data (Min-Max scaled between 0 and 1):\n", scaled_data)
print("\nMin Values for each feature:\n", min_vals)
print("\nMax Values for each feature:\n", max_vals)


Scaled Data (Min-Max scaled between 0 and 1):
 [[0.000000e+00 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [5.000025e-06 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [1.000005e-05 0.000000e+00 8.000000e-04 ... 1.800000e-03 1.450000e-03
  2.000000e-03]
 ...
 [9.999900e-01 1.000000e-04 1.100000e-03 ... 4.500000e-03 1.550000e-03
  1.000000e-03]
 [9.999950e-01 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [1.000000e+00 2.000000e-04 8.000000e-04 ... 2.300000e-03 2.150000e-03
  0.000000e+00]]

Min Values for each feature:
 [ 0.00000000e+00 -9.53674316e-07 -9.53674316e-06 -4.29153442e-05
 -1.14440918e-04 -2.00271606e-04 -2.40325928e-04 -2.00271606e-04
 -1.14440918e-04 -4.29153442e-05 -9.53674316e-06 -9.53674316e-07
 -9.53674316e-06 -8.58306885e-05 -3.43322754e-04 -8.01086426e-04
 -1.20162964e-03 -1.20162964e-03 -8.01086426e-04 -3.43322754e-04
 -8.58306885e-05 -9.53674316e-06 -4.29153442e-05 -3.43322754e-04
 -1

In [10]:

# import numpy as np

# # Convert a list of lists to a NumPy array
# X_train = np.array(train)
# # Assuming X_train is your training data
# # X_train should be a 2D NumPy array

# # Initialize an empty array to store the scaled data
# X_train_scaled = np.zeros_like(X_train)


# # Iterate through each row of X_train
# for i in range(X_train.shape[0]):
#     row = X_train[i, :]  # Get the current row
#     min_val = np.min(row)  # Calculate the minimum value in the row
#     max_val = np.max(row)  # Calculate the maximum value in the row

#     # Avoid division by zero
#     if max_val != min_val:
#         scaled_row = (row - min_val) / (max_val - min_val)
#     else:
#         scaled_row = np.zeros_like(row)  # If max and min are the same, set all values to 0
#          # Debugging: Print scaled_row for each row
# #     print(f"Scaled Row {i}: {scaled_row}")

#     X_train_scaled[i, :] = scaled_row  # Store the scaled row in the result array

# Now, X_train_scaled contains your normalized training data with each row scaled individually between 0 and 1



In [21]:
# X_train_scaled

In [16]:
# print(X_train_scaled)

In [12]:
# import numpy as np

# # Assuming X_train_scaled is your scaled dataset
# # Create an empty array for the formatted data
# X_train_formatted = np.empty_like(X_train_scaled, dtype=np.float64)

# # Iterate through each row and column of X_train_scaled
# for i in range(X_train_scaled.shape[0]):
#     for j in range(X_train_scaled.shape[1]):
#         scaled_value = X_train_scaled[i, j]
#         formatted_value = "{:.5f}".format(scaled_value)
#         X_train_formatted[i, j] = float(formatted_value)

# Now, X_train_formatted contains your scaled dataset with all values formatted as decimals


In [17]:
# print(X_train_formatted)

In [23]:
# Convert a list of lists to a NumPy array
# X_test = np.array(test)

In [24]:
# X_test

In [25]:
# X_train = X_train.astype(np.float64)
# X_test = X_test.astype(np.float64)


In [26]:
# X_test

In [ ]:
# import numpy as np

# # Assuming X_train is your training data (a 2D NumPy array)
# # and X_test is your test data (a 2D NumPy array)

# # Initialize an empty array for scaled test data
# X_test_scaled = np.zeros_like(X_test)

# # Iterate through each row of X_test
# for i in range(X_test.shape[0]):
#     row = X_test[i, :]  # Get the current row
#     min_val = np.min(X_train[i, :])  # Calculate the minimum value in the corresponding training data row
#     max_val = np.max(X_train[i, :])  # Calculate the maximum value in the corresponding training data row

#     # Scale the row to have values between 0 and 1 using training data's min and max
#     if max_val != min_val:
#         scaled_row = (row - min_val) / (max_val - min_val)
#     else:
#         scaled_row = np.zeros_like(row)  # If max and min are the same, set all values to 0

#     X_test_scaled[i, :] = scaled_row  # Store the scaled row in the result array

# # Now, X_test_scaled contains your scaled test data with each row scaled using training data's min and max



In [27]:
# X_test_scaled

In [28]:
# X_train_formatted.shape

In [ ]:
                                                  #  PCA of train data

In [12]:
train

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
0,0,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000043,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07
1,1,-9.536743e-07,-9.536743e-06,-0.000043,0.000886,-0.000200,0.000760,-0.000200,-0.000114,-0.000043,...,-0.000043,-0.000086,-0.000043,0.000914,0.000914,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07
2,2,-9.536743e-07,-1.536743e-06,0.000007,0.000129,0.000268,0.000270,0.000243,0.000125,0.000001,...,0.000042,0.000084,0.000048,0.000081,0.000106,0.000072,1.046326e-05,0.000008,0.000019,1.046326e-06
3,3,4.632568e-08,-5.536743e-06,0.000012,0.000245,0.000492,0.000522,0.000396,0.000197,-0.000003,...,0.000068,0.000151,0.000100,0.000180,0.000202,0.000153,2.146326e-05,0.000015,0.000046,-9.536743e-07
4,4,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000043,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199995,-9.536743e-07,4.632568e-07,-0.000003,0.000176,0.000350,0.000290,0.000200,0.000206,-0.000023,...,0.000017,0.000124,0.000057,0.000104,0.000144,0.000027,4.632568e-07,0.000060,0.000020,-9.536743e-07
199996,199996,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000043,-0.000086,-0.000043,0.000914,0.000914,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07
199997,199997,4.632568e-08,1.463257e-06,-0.000005,-0.000031,-0.000019,-0.000037,-0.000037,-0.000015,-0.000005,...,0.000028,0.000115,0.000131,0.000110,0.000213,0.000094,1.646326e-05,0.000035,0.000021,4.632568e-08
199998,199998,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000043,-0.000086,-0.000043,0.001914,-0.000086,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07


In [22]:
# target_column = train['target']
# train=train.drop(columns=['target']) 

In [18]:
target_column

0           Streptococcus_pyogenes
1              Salmonella_enterica
2              Salmonella_enterica
3              Salmonella_enterica
4               Enterococcus_hirae
                    ...           
199995         Salmonella_enterica
199996      Streptococcus_pyogenes
199997    Streptococcus_pneumoniae
199998       Staphylococcus_aureus
199999       Klebsiella_pneumoniae
Name: target, Length: 200000, dtype: object

In [19]:
# train

In [20]:
scaled_data

array([[0.000000e+00, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [5.000025e-06, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [1.000005e-05, 0.000000e+00, 8.000000e-04, ..., 1.800000e-03,
        1.450000e-03, 2.000000e-03],
       ...,
       [9.999900e-01, 1.000000e-04, 1.100000e-03, ..., 4.500000e-03,
        1.550000e-03, 1.000000e-03],
       [9.999950e-01, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [1.000000e+00, 2.000000e-04, 8.000000e-04, ..., 2.300000e-03,
        2.150000e-03, 0.000000e+00]])

In [21]:
import numpy as np
from sklearn.decomposition import PCA

# Assuming X_train_scaled is your scaled training data
# You can use X_train_formatted if you've converted it to human-readable format X_train_formatted

# Initialize the PCA model with the desired number of components
n_components = 33  # Adjust this value as needed
pca = PCA(n_components=n_components)

# Fit the PCA model to your scaled data
pca.fit(scaled_data)

# Transform the data to its principal components
X_train_pca = pca.transform(scaled_data)
# Create a DataFrame for the PCA results using common columns
pca_df = pd.DataFrame(data=X_train_pca, columns=[f'PC{i}' for i in range(1, 34)])
# Printing the PCA results
print(pca_df)

# Now, X_train_pca contains your training data reduced to the specified number of principal components
# You can use X_train_pca for classification

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


             PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
0      -0.195312  1.450241 -0.493021 -0.572659 -0.297635  0.197050  0.409117   
1      -0.455772  0.023449 -0.499954 -0.147320  0.226929  0.058249 -0.082594   
2      -0.455364 -0.176181 -0.501369  0.009695 -0.088106 -0.045754 -0.005835   
3      -0.514005 -0.202394 -0.502223  0.088732 -0.101303  0.066163 -0.007517   
4       0.498484  1.020506 -0.501142  0.310237 -0.013752 -0.050951 -0.443789   
...          ...       ...       ...       ...       ...       ...       ...   
199995 -0.481202 -0.183816  0.498148  0.051616 -0.089788 -0.002521 -0.010074   
199996 -0.196881 -0.019007  0.500695 -0.205747  0.337803 -0.063218 -0.172581   
199997 -0.212165 -0.158487  0.500078 -0.081012 -0.065430 -0.176712 -0.001405   
199998 -0.364806  0.009943  0.501534 -0.485663  0.894587  0.268230  0.036807   
199999 -0.871026 -0.253236  0.495066  0.280512 -0.152346  0.362412 -0.034243   

             PC8       PC9      PC10  .

In [ ]:
                   # pca of test data

In [30]:
# import numpy as np
# from sklearn.decomposition import PCA

# # Assuming X_test_scaled is your scaled training data
# # You can use X_test_formatted if you've converted it to human-readable format X_test_formatted

# # Initialize the PCA model with the desired number of components
# n_components = 33  # Adjust this value as needed
# pca = PCA(n_components=n_components)

# # Fit the PCA model to your scaled data
# pca.fit(X_test_scaled)

# # Transform the data to its principal components
# pca_df_test = pca.transform(X_test_scaled)
# # Create a DataFrame for the PCA results using common columns
# pca_df_test = pd.DataFrame(pca_df_test, columns=[f'PC{i}' for i in range(1, 34)])
# # Printing the PCA results
# print(pca_df_test)

In [31]:
# Assuming you have a target_column, and pca_df is your DataFrame
# target_column
# Add the target_column to pca_df
# pca_df['target'] = target_column


In [22]:
print(pca_df)

             PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
0      -0.195312  1.450241 -0.493021 -0.572659 -0.297635  0.197050  0.409117   
1      -0.455772  0.023449 -0.499954 -0.147320  0.226929  0.058249 -0.082594   
2      -0.455364 -0.176181 -0.501369  0.009695 -0.088106 -0.045754 -0.005835   
3      -0.514005 -0.202394 -0.502223  0.088732 -0.101303  0.066163 -0.007517   
4       0.498484  1.020506 -0.501142  0.310237 -0.013752 -0.050951 -0.443789   
...          ...       ...       ...       ...       ...       ...       ...   
199995 -0.481202 -0.183816  0.498148  0.051616 -0.089788 -0.002521 -0.010074   
199996 -0.196881 -0.019007  0.500695 -0.205747  0.337803 -0.063218 -0.172581   
199997 -0.212165 -0.158487  0.500078 -0.081012 -0.065430 -0.176712 -0.001405   
199998 -0.364806  0.009943  0.501534 -0.485663  0.894587  0.268230  0.036807   
199999 -0.871026 -0.253236  0.495066  0.280512 -0.152346  0.362412 -0.034243   

             PC8       PC9      PC10  .

In [23]:
pca_df.shape

(200000, 33)

In [25]:
# pca_df
X = pca_df  # Extract all feature columns except 'target'
y = target_column

In [24]:
target_column

0           Streptococcus_pyogenes
1              Salmonella_enterica
2              Salmonella_enterica
3              Salmonella_enterica
4               Enterococcus_hirae
                    ...           
199995         Salmonella_enterica
199996      Streptococcus_pyogenes
199997    Streptococcus_pneumoniae
199998       Staphylococcus_aureus
199999       Klebsiella_pneumoniae
Name: target, Length: 200000, dtype: object

In [26]:
X

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33
0,-0.195312,1.450241,-0.493021,-0.572659,-0.297635,0.197050,0.409117,-0.055661,0.132696,-0.020633,...,0.022853,-0.034065,-0.049979,-0.063715,0.009424,0.024634,-0.045399,-0.011842,-0.021832,-0.064831
1,-0.455772,0.023449,-0.499954,-0.147320,0.226929,0.058249,-0.082594,-0.255567,-0.024214,-0.060199,...,-0.025985,-0.012940,-0.040103,0.056004,-0.008834,-0.070801,-0.012537,0.058317,0.019378,-0.038456
2,-0.455364,-0.176181,-0.501369,0.009695,-0.088106,-0.045754,-0.005835,0.039792,-0.008607,-0.046811,...,-0.001723,0.003601,0.003857,-0.002821,-0.002350,0.000247,0.001112,0.007009,-0.000284,0.000803
3,-0.514005,-0.202394,-0.502223,0.088732,-0.101303,0.066163,-0.007517,0.020169,0.002250,0.002056,...,0.002255,0.009617,-0.007120,-0.003542,-0.005330,0.006924,-0.009082,-0.000051,-0.004132,0.006491
4,0.498484,1.020506,-0.501142,0.310237,-0.013752,-0.050951,-0.443789,0.146972,0.394156,0.100479,...,-0.060237,0.036303,0.199551,-0.015065,-0.114270,-0.079333,0.049267,-0.011205,0.061179,0.051704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,-0.481202,-0.183816,0.498148,0.051616,-0.089788,-0.002521,-0.010074,0.040612,-0.002718,-0.023120,...,-0.000451,-0.005653,-0.006765,0.003439,0.003325,0.001006,-0.000084,0.001114,0.003677,0.006985
199996,-0.196881,-0.019007,0.500695,-0.205747,0.337803,-0.063218,-0.172581,-0.316241,-0.015530,-0.071131,...,-0.056091,-0.005123,0.020191,0.018326,0.011389,-0.012123,0.038576,-0.023561,0.004369,0.002474
199997,-0.212165,-0.158487,0.500078,-0.081012,-0.065430,-0.176712,-0.001405,0.050292,-0.012731,-0.063049,...,-0.000196,-0.004304,0.014964,0.001011,-0.000241,-0.007622,0.010461,0.010078,0.002753,-0.002383
199998,-0.364806,0.009943,0.501534,-0.485663,0.894587,0.268230,0.036807,0.354251,0.002307,0.034540,...,0.012271,-0.000127,0.008678,-0.005181,-0.011460,0.023772,-0.023821,0.005354,-0.014740,0.023635


In [27]:
y

0           Streptococcus_pyogenes
1              Salmonella_enterica
2              Salmonella_enterica
3              Salmonella_enterica
4               Enterococcus_hirae
                    ...           
199995         Salmonella_enterica
199996      Streptococcus_pyogenes
199997    Streptococcus_pneumoniae
199998       Staphylococcus_aureus
199999       Klebsiella_pneumoniae
Name: target, Length: 200000, dtype: object

In [28]:
from sklearn.model_selection import train_test_split

X_train_new, X_val, y_train_new, y_val = train_test_split(X, y, test_size=0.1, random_state=42)


In [29]:
X_train_new.shape

(180000, 33)

In [30]:
X_val

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33
119737,-0.386470,-0.182775,0.097533,0.019170,-0.081876,-0.061948,0.015088,0.036640,0.011090,-0.006318,...,0.031962,0.001317,0.000929,-0.000437,0.008540,0.004626,-0.001166,-0.015511,-0.014988,-0.003926
72272,-0.105635,-0.159313,-0.138383,-0.071882,-0.068798,-0.080831,-0.012779,0.050981,-0.028005,-0.116536,...,0.000048,-0.011507,0.002599,0.011756,-0.000979,-0.011044,0.019457,0.000982,0.008589,-0.005906
158154,0.175043,-0.177288,0.292008,-0.097076,-0.046034,-0.155100,0.003885,0.039681,-0.000279,-0.027084,...,-0.005959,0.000899,0.006707,-0.004270,-0.015735,0.002599,0.003049,-0.007224,0.001811,0.002649
65426,-0.389168,-0.183872,-0.174027,0.017031,-0.083768,-0.064640,0.019399,0.033571,0.007549,-0.000463,...,0.023846,0.002622,-0.002832,0.000865,0.001688,0.001568,-0.003126,-0.013896,-0.012058,-0.009572
30074,-0.507506,0.098146,-0.353935,0.326891,-0.259413,0.228391,-0.064639,-0.036469,-0.295749,0.295501,...,-0.226465,0.098249,-0.154382,-0.123414,-0.087712,-0.033500,0.186009,0.048538,0.157973,-0.023531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193188,-0.196705,-0.154964,0.466160,-0.086962,-0.063075,-0.162100,0.002073,0.052265,-0.020806,-0.077996,...,-0.009308,-0.007170,0.001600,0.002653,-0.000991,-0.011181,0.014245,0.003337,0.003712,-0.012623
35956,-0.417475,-0.183279,-0.322112,0.081710,0.004382,-0.053517,0.079085,-0.094336,0.006505,0.076265,...,0.071428,0.089642,-0.040043,0.042594,-0.011574,-0.043051,0.032918,-0.002719,-0.010835,0.002037
149399,-0.315960,-0.052447,0.245334,0.231740,-0.322613,-0.098963,0.086759,0.344536,-0.476490,0.083378,...,0.265437,0.185058,-0.020303,-0.121947,0.149085,-0.117611,-0.041608,0.109438,0.030327,0.008249
97701,0.016852,-0.168015,-0.010703,-0.092272,-0.053644,-0.159632,0.001139,0.046004,-0.008243,-0.049238,...,-0.005137,-0.002077,0.008148,-0.004469,-0.012784,-0.001383,0.003051,-0.000997,0.002039,-0.001446


In [31]:
X_train_new

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33
38762,-0.387754,-0.179544,-0.307261,0.005502,-0.083125,-0.075855,0.013105,0.037369,0.003686,-0.013009,...,0.019375,0.002832,-0.004338,-0.001978,0.001167,0.000759,0.001539,-0.011899,-0.012244,-0.010960
76883,-0.447848,-0.157481,-0.116366,-0.052780,-0.074742,-0.113374,-0.005894,0.059995,-0.023750,-0.088320,...,-0.009947,-0.008838,0.000467,0.003741,0.004839,-0.011771,0.020994,0.007224,0.007783,-0.008084
2018,0.174330,-0.176383,-0.488638,-0.106370,-0.037568,-0.149143,0.007158,0.039391,0.002777,-0.026415,...,-0.003014,-0.002793,-0.009720,0.003231,-0.009156,0.005211,-0.001288,-0.008947,0.004727,0.000822
133899,1.052958,-0.247346,0.172211,-0.018864,-0.111708,0.378535,-0.055296,0.011147,0.020262,0.001541,...,0.016233,-0.008901,-0.002986,0.010563,0.015419,-0.017168,-0.003350,-0.006343,0.013677,-0.011666
170373,-0.531421,-0.007551,0.352724,-0.450606,0.890268,0.287516,0.035857,0.342573,-0.022159,0.056706,...,0.006478,0.023627,0.012857,-0.024299,-0.000357,0.006838,0.011784,-0.001417,-0.014321,-0.013619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,0.040257,1.326085,0.101818,-0.244504,-0.204954,-0.000720,-0.516811,0.044313,-0.321465,0.138502,...,-0.111737,-0.039275,-0.026533,0.110822,0.118313,-0.015854,0.004991,0.177177,-0.094105,0.010459
103694,-0.430906,-0.170642,0.017309,0.001839,-0.084718,-0.071504,0.004567,0.050981,-0.009432,-0.027497,...,0.004676,-0.003527,-0.004970,0.003683,0.003081,0.005589,0.000009,-0.007730,-0.000199,-0.009041
131932,0.270503,-0.192393,0.160677,-0.049149,-0.092770,0.083745,-0.027371,0.039421,-0.004432,-0.055591,...,0.008553,-0.006253,0.005661,0.004572,0.006198,-0.010724,0.008576,0.005099,0.005388,-0.011558
146867,0.596427,-0.217094,0.236906,-0.087065,-0.000230,-0.083391,0.056255,0.009120,-0.016778,0.021120,...,-0.114646,-0.062275,-0.043891,-0.089426,-0.014326,-0.002734,-0.035707,-0.020839,0.015417,0.040340


In [32]:
X_val.shape

(20000, 33)

In [33]:
y_train_new.shape

(180000,)

In [34]:
y_val

119737    Escherichia_fergusonii
72272      Staphylococcus_aureus
158154    Streptococcus_pyogenes
65426     Escherichia_fergusonii
30074      Klebsiella_pneumoniae
                   ...          
193188    Streptococcus_pyogenes
35956           Escherichia_coli
149399          Escherichia_coli
97701     Streptococcus_pyogenes
51732       Campylobacter_jejuni
Name: target, Length: 20000, dtype: object

In [35]:
X_train_new

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33
38762,-0.387754,-0.179544,-0.307261,0.005502,-0.083125,-0.075855,0.013105,0.037369,0.003686,-0.013009,...,0.019375,0.002832,-0.004338,-0.001978,0.001167,0.000759,0.001539,-0.011899,-0.012244,-0.010960
76883,-0.447848,-0.157481,-0.116366,-0.052780,-0.074742,-0.113374,-0.005894,0.059995,-0.023750,-0.088320,...,-0.009947,-0.008838,0.000467,0.003741,0.004839,-0.011771,0.020994,0.007224,0.007783,-0.008084
2018,0.174330,-0.176383,-0.488638,-0.106370,-0.037568,-0.149143,0.007158,0.039391,0.002777,-0.026415,...,-0.003014,-0.002793,-0.009720,0.003231,-0.009156,0.005211,-0.001288,-0.008947,0.004727,0.000822
133899,1.052958,-0.247346,0.172211,-0.018864,-0.111708,0.378535,-0.055296,0.011147,0.020262,0.001541,...,0.016233,-0.008901,-0.002986,0.010563,0.015419,-0.017168,-0.003350,-0.006343,0.013677,-0.011666
170373,-0.531421,-0.007551,0.352724,-0.450606,0.890268,0.287516,0.035857,0.342573,-0.022159,0.056706,...,0.006478,0.023627,0.012857,-0.024299,-0.000357,0.006838,0.011784,-0.001417,-0.014321,-0.013619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,0.040257,1.326085,0.101818,-0.244504,-0.204954,-0.000720,-0.516811,0.044313,-0.321465,0.138502,...,-0.111737,-0.039275,-0.026533,0.110822,0.118313,-0.015854,0.004991,0.177177,-0.094105,0.010459
103694,-0.430906,-0.170642,0.017309,0.001839,-0.084718,-0.071504,0.004567,0.050981,-0.009432,-0.027497,...,0.004676,-0.003527,-0.004970,0.003683,0.003081,0.005589,0.000009,-0.007730,-0.000199,-0.009041
131932,0.270503,-0.192393,0.160677,-0.049149,-0.092770,0.083745,-0.027371,0.039421,-0.004432,-0.055591,...,0.008553,-0.006253,0.005661,0.004572,0.006198,-0.010724,0.008576,0.005099,0.005388,-0.011558
146867,0.596427,-0.217094,0.236906,-0.087065,-0.000230,-0.083391,0.056255,0.009120,-0.016778,0.021120,...,-0.114646,-0.062275,-0.043891,-0.089426,-0.014326,-0.002734,-0.035707,-0.020839,0.015417,0.040340


In [36]:
X_train_new['target'] = y_train_new


In [37]:
X_train_new

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,target
38762,-0.387754,-0.179544,-0.307261,0.005502,-0.083125,-0.075855,0.013105,0.037369,0.003686,-0.013009,...,0.002832,-0.004338,-0.001978,0.001167,0.000759,0.001539,-0.011899,-0.012244,-0.010960,Escherichia_fergusonii
76883,-0.447848,-0.157481,-0.116366,-0.052780,-0.074742,-0.113374,-0.005894,0.059995,-0.023750,-0.088320,...,-0.008838,0.000467,0.003741,0.004839,-0.011771,0.020994,0.007224,0.007783,-0.008084,Klebsiella_pneumoniae
2018,0.174330,-0.176383,-0.488638,-0.106370,-0.037568,-0.149143,0.007158,0.039391,0.002777,-0.026415,...,-0.002793,-0.009720,0.003231,-0.009156,0.005211,-0.001288,-0.008947,0.004727,0.000822,Streptococcus_pyogenes
133899,1.052958,-0.247346,0.172211,-0.018864,-0.111708,0.378535,-0.055296,0.011147,0.020262,0.001541,...,-0.008901,-0.002986,0.010563,0.015419,-0.017168,-0.003350,-0.006343,0.013677,-0.011666,Campylobacter_jejuni
170373,-0.531421,-0.007551,0.352724,-0.450606,0.890268,0.287516,0.035857,0.342573,-0.022159,0.056706,...,0.023627,0.012857,-0.024299,-0.000357,0.006838,0.011784,-0.001417,-0.014321,-0.013619,Klebsiella_pneumoniae
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,0.040257,1.326085,0.101818,-0.244504,-0.204954,-0.000720,-0.516811,0.044313,-0.321465,0.138502,...,-0.039275,-0.026533,0.110822,0.118313,-0.015854,0.004991,0.177177,-0.094105,0.010459,Streptococcus_pneumoniae
103694,-0.430906,-0.170642,0.017309,0.001839,-0.084718,-0.071504,0.004567,0.050981,-0.009432,-0.027497,...,-0.003527,-0.004970,0.003683,0.003081,0.005589,0.000009,-0.007730,-0.000199,-0.009041,Escherichia_coli
131932,0.270503,-0.192393,0.160677,-0.049149,-0.092770,0.083745,-0.027371,0.039421,-0.004432,-0.055591,...,-0.006253,0.005661,0.004572,0.006198,-0.010724,0.008576,0.005099,0.005388,-0.011558,Campylobacter_jejuni
146867,0.596427,-0.217094,0.236906,-0.087065,-0.000230,-0.083391,0.056255,0.009120,-0.016778,0.021120,...,-0.062275,-0.043891,-0.089426,-0.014326,-0.002734,-0.035707,-0.020839,0.015417,0.040340,Enterococcus_hirae


In [40]:
                ##       K-MEAN
import pandas as pd
import cupy as cp  # Assuming you have CuPy installed
from cuml.cluster import KMeans as cuKMeans  # Assuming you have cuML installed

# Assuming you have your data in a DataFrame called df
# Features used for clustering (adjust as needed)
# features_for_clustering = ['feature1', 'feature2', 'feature3']

# Number of clusters (adjust as needed)
num_clusters = 1000

# Assuming you have a DataFrame called pca_df
# Fit K-means model using cuML
kmeans = cuKMeans(n_clusters=num_clusters, random_state=0, n_init=100)
X_train_new['cluster_id'] = kmeans.fit_predict(cp.asarray(X_train_new.drop('target', axis=1))).get()

# Initialize a new DataFrame for the optimized data
new_df = pd.DataFrame()

# Loop through each cluster
for cluster_id in range(num_clusters):
    each_cluster = X_train_new[X_train_new['cluster_id'].values == cluster_id]
    
    # Check if the cluster is not empty
    if not each_cluster.empty:
        # Calculate the predominant class in the cluster
        predominant_class = each_cluster['target'].value_counts().idxmax()
        
        # Filter records of the predominant class
        each_cluster_filtered = each_cluster[each_cluster['target'] == predominant_class]
        
        # Calculate the average of the records in the cluster
        averaged_record = each_cluster_filtered.mean(numeric_only=True)
        
        # Assign the predominant class to the averaged record
        averaged_record['target'] = predominant_class
        
        # Append the averaged record to the new DataFrame
        new_df = pd.concat([new_df, averaged_record.to_frame().transpose()], ignore_index=True)

# Display the new DataFrame
# print(new_df)


In [41]:
new_df.shape

(1000, 35)

In [42]:
new_df

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,cluster_id,target
0,0.785976,0.50309,0.198195,0.482415,0.246652,-0.040396,0.217061,0.015569,-0.181787,-0.090103,...,0.100143,0.15386,-0.067116,0.050709,0.027281,-0.077669,0.102086,-0.067866,0.0,Staphylococcus_aureus
1,0.532899,0.496166,0.169969,0.462912,0.2746,-0.257008,0.284317,-0.023628,-0.198051,-0.057232,...,0.075158,0.048967,0.05598,-0.061085,-0.222276,-0.043654,0.055269,-0.075333,1.0,Streptococcus_pyogenes
2,0.082737,-0.066362,-0.100283,-0.081661,0.187028,-0.127311,-0.023701,-0.260229,0.049308,0.013699,...,0.022388,0.006293,0.013302,-0.007414,-0.004992,-0.014726,0.013096,0.008839,2.0,Streptococcus_pneumoniae
3,-0.008497,0.42385,0.161494,0.569572,0.358063,-0.259363,0.223901,0.081972,-0.151949,-0.139223,...,-0.061519,-0.109166,0.001818,-0.252313,-0.062928,0.082873,0.080653,-0.036103,3.0,Escherichia_coli
4,-0.268554,0.876711,-0.138526,0.588051,0.031966,-0.019329,-0.377922,0.160385,0.414027,0.031031,...,-0.104005,0.086334,-0.163204,-0.192725,0.18161,0.005344,0.17716,-0.0266,4.0,Klebsiella_pneumoniae
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.730362,-0.104258,0.031437,-0.070826,-0.047016,0.044075,0.027851,-0.010321,-0.042574,0.230815,...,-0.22262,0.235124,-0.118417,-0.056489,-0.267788,0.003453,-0.034366,-0.071991,995.0,Enterococcus_hirae
996,-0.210352,0.166974,-0.307955,0.248888,-0.063712,-0.052602,0.178131,-0.012154,-0.019916,0.314165,...,0.118524,0.114389,-0.231321,-0.276458,-0.137625,-0.087217,0.000994,-0.047622,996.0,Escherichia_coli
997,-0.495831,-0.147776,-0.017177,0.124999,-0.108715,-0.087134,-0.018306,0.079046,-0.098513,-0.060328,...,0.051045,-0.037086,0.006443,-0.029451,0.076969,-0.008335,-0.088125,-0.052473,997.0,Salmonella_enterica
998,0.189642,-0.115956,-0.07823,-0.111868,0.109624,-0.19977,-0.057725,-0.171625,-0.037911,0.249202,...,0.071815,-0.119955,-0.058446,-0.195442,0.125183,-0.004472,0.148781,-0.080353,998.0,Streptococcus_pneumoniae


In [43]:
X_1 = new_df.drop(columns=['target','cluster_id'])  # Extract all feature columns except 'target'
y_1 = new_df['target']

In [89]:
X_1

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33
0,0.785976,0.50309,0.198195,0.482415,0.246652,-0.040396,0.217061,0.015569,-0.181787,-0.090103,...,0.189049,-0.216528,0.100143,0.15386,-0.067116,0.050709,0.027281,-0.077669,0.102086,-0.067866
1,0.532899,0.496166,0.169969,0.462912,0.2746,-0.257008,0.284317,-0.023628,-0.198051,-0.057232,...,0.017982,0.050963,0.075158,0.048967,0.05598,-0.061085,-0.222276,-0.043654,0.055269,-0.075333
2,0.082737,-0.066362,-0.100283,-0.081661,0.187028,-0.127311,-0.023701,-0.260229,0.049308,0.013699,...,0.012807,-0.001718,0.022388,0.006293,0.013302,-0.007414,-0.004992,-0.014726,0.013096,0.008839
3,-0.008497,0.42385,0.161494,0.569572,0.358063,-0.259363,0.223901,0.081972,-0.151949,-0.139223,...,0.078126,-0.097074,-0.061519,-0.109166,0.001818,-0.252313,-0.062928,0.082873,0.080653,-0.036103
4,-0.268554,0.876711,-0.138526,0.588051,0.031966,-0.019329,-0.377922,0.160385,0.414027,0.031031,...,-0.076328,0.197198,-0.104005,0.086334,-0.163204,-0.192725,0.18161,0.005344,0.17716,-0.0266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.730362,-0.104258,0.031437,-0.070826,-0.047016,0.044075,0.027851,-0.010321,-0.042574,0.230815,...,-0.070765,-0.103475,-0.22262,0.235124,-0.118417,-0.056489,-0.267788,0.003453,-0.034366,-0.071991
996,-0.210352,0.166974,-0.307955,0.248888,-0.063712,-0.052602,0.178131,-0.012154,-0.019916,0.314165,...,-0.118009,-0.123828,0.118524,0.114389,-0.231321,-0.276458,-0.137625,-0.087217,0.000994,-0.047622
997,-0.495831,-0.147776,-0.017177,0.124999,-0.108715,-0.087134,-0.018306,0.079046,-0.098513,-0.060328,...,-0.012696,-0.188808,0.051045,-0.037086,0.006443,-0.029451,0.076969,-0.008335,-0.088125,-0.052473
998,0.189642,-0.115956,-0.07823,-0.111868,0.109624,-0.19977,-0.057725,-0.171625,-0.037911,0.249202,...,-0.00539,0.106479,0.071815,-0.119955,-0.058446,-0.195442,0.125183,-0.004472,0.148781,-0.080353


In [87]:
y_1

0         Staphylococcus_aureus
1        Streptococcus_pyogenes
2      Streptococcus_pneumoniae
3              Escherichia_coli
4         Klebsiella_pneumoniae
                 ...           
995          Enterococcus_hirae
996            Escherichia_coli
997         Salmonella_enterica
998    Streptococcus_pneumoniae
999            Escherichia_coli
Name: target, Length: 1000, dtype: object

In [25]:
# from sklearn.model_selection import train_test_split

# X_trainnew, X_val, y_trainnew, y_val = train_test_split(X, y, test_size=0.3, random_state=42)


In [56]:
# y_trainnew

In [55]:
# y_val

In [57]:
# X_trainnew

In [58]:
# X_val   X_val, , y_val,X_1, y_1

In [ ]:
# X_val,  y_val

In [106]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

  # Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators = 30, max_depth = 6)  # You can adjust n_estimators
 # You can adjust n_estimators
# Train the Random Forest Model
rf_classifier.fit(X_1, y_1)
# Step 4: Model Evaluation
# y_pred = rf_classifier.predict(pca_df_test)  # Predict on Test Data
y_pred = rf_classifier.predict(X_val)  # Predict on Test Data

accuracy = accuracy_score(y_val, y_pred,)
accuracy_percent = accuracy * 100
print(f"Accuracy: {accuracy_percent:.2f}%")

Accuracy: 52.55%


In [105]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

  # Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier( n_estimators=30, max_depth=6)  # You can adjust n_estimators
 # You can adjust n_estimators
# Train the Random Forest Model
rf_classifier.fit(X_1, y_1)
# Step 4: Model Evaluation
# y_pred = rf_classifier.predict(pca_df_test)  # Predict on Test Data
y_pred = rf_classifier.predict(X_1)  # Predict on Test Data

accuracy = accuracy_score(y_1, y_pred,)
accuracy_percent = accuracy * 100
print(f"Accuracy: {accuracy_percent:.2f}%")

Accuracy: 90.60%


In [ ]:
# X_val, , y_val,X_1, y_1

In [58]:
 ## 2. AdaBoost Algorithm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
 # Step 3: Initialize the base estimator (Decision Tree)
base_estimator = DecisionTreeClassifier(max_depth=3)  # You can adjust max_depth
 # Step 4: Initialize the AdaBoost Classifier
adaboost_classifier = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=50,random_state=32)
# You can adjust n_estimators (number of weak learners)max_depth=3,random_state=32
 # Step 5: Train the AdaBoost Model
adaboost_classifier.fit(X_1, y_1)
 # Step 6: Model Evaluation
y_pred = adaboost_classifier.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
# report = classification_report(y_true, y_pred,zero_division=1)

accuracy_percent = accuracy * 100
print(f"Accuracy_train: {accuracy_percent:.2f}%")

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy_train: 47.98%


In [67]:
 ## 2. AdaBoost Algorithm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
 # Step 3: Initialize the base estimator (Decision Tree)
base_estimator = DecisionTreeClassifier(max_depth=3)  # You can adjust max_depth
 # Step 4: Initialize the AdaBoost Classifier
adaboost_classifier = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=50,random_state=32)
# You can adjust n_estimators (number of weak learners)max_depth=3,random_state=32
 # Step 5: Train the AdaBoost Model
adaboost_classifier.fit(X_1, y_1)
 # Step 6: Model Evaluation
y_pred = adaboost_classifier.predict(X_1)
accuracy_test_2 = accuracy_score(y_1, y_pred)
# report = classification_report(y_true, y_pred,zero_division=1)

accuracy_percent_test = accuracy_test_2 * 100
print(f"Accuracy: {accuracy_percent_test:.2f}%")

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy: 73.50%


In [ ]:
# X_val, , y_val,X_1, y_1

In [68]:
# 3. ************  Naive Bayes Algorithm  ************
#     for test
from sklearn.preprocessing import StandardScaler

from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
# Create a pipeline with MinMaxScaler and Gaussian Naive Bayes
pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Change the scaler
        ('classifier', GaussianNB())  # Gaussian Naive Bayes classifier
])

# Train the model using the pipeline
pipeline.fit(X_1, y_1)

# Make predictions on the test set
y_pred = pipeline.predict(X_1)

# Evaluate the model
accuracy = accuracy_score(y_1, y_pred)

accuracy_percent = accuracy * 100
print(f"Accuracy_3: {accuracy_percent:.2f}%")

Accuracy_3: 68.50%


In [70]:
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
# Step 2: Create and train the Gaussian Naive Bayes model
naive_bayes_model = GaussianNB()
naive_bayes_model.fit(X_1, y_1)

# Step 3: Make predictions on the test set
y_pred_test = naive_bayes_model.predict(X_val)

# Step 4: Evaluate the model
test_accuracy = accuracy_score(y_val, y_pred_test)

accuracy_percent = test_accuracy * 100
print(f"Accuracy_3: {accuracy_percent:.2f}%")

Accuracy_3: 43.75%


In [ ]:
# X_val   y_val,

In [136]:
## 4. SVM - Support Vector Machines code

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Initialize the Support Vector Machine classifier
# You can adjust the kernel (linear, polynomial, radial basis function, etc.) and other hyperparameters

# svm_classifier = SVC(kernel='linear', random_state=42,C=1.0)
svm_classifier = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
# svm_classifier = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)


# Train the classifier on the training data
svm_classifier.fit(X_1, y_1)

# Make predictions on the test data
y_pred = svm_classifier.predict(X_val)

# Evaluate the model
# accuracy_4 = accuracy_score(y_1, y_pred)
accuracy_4 = accuracy_score(y_val, y_pred)

accuracy_percent_4 = accuracy_4 * 100
print(f"Accuracy: {accuracy_percent_4:.2f}%") 

Accuracy: 51.59%


In [143]:

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Initialize the Support Vector Machine classifier
# You can adjust the kernel (linear, polynomial, radial basis function, etc.) and other hyperparameters

# svm_classifier = SVC(kernel='linear', random_state=42,C=1.0)
svm_classifier = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)

# svm_classifier = SVC(kernel='rbf', C=1.0, gamma='auto', random_state=32)


# Train the classifier on the training data
svm_classifier.fit(X_1, y_1)

# Make predictions on the test data
y_pred = svm_classifier.predict(X_1)

# Evaluate the model
# accuracy_4 = accuracy_score(y_1, y_pred)
accuracy = accuracy_score(y_1, y_pred)

accuracy_percent = accuracy * 100
print(f"Accuracy: {accuracy_percent:.2f}%") 

Accuracy: 88.40%


In [ ]:
                          # Agorithms

In [ ]:
#Using SVM algorithm
import numpy as np
from sklearn.svm import SVC

In [ ]:

X_train = cluster_centers_df
y_train = clustered_y_train  

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
# Create a pipeline with MinMax scaling and Naive Bayes classifier
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),  # Scale features to the [0, 1] range
    ('classifier', GaussianNB())  # Use Gaussian Naive Bayes classifier
])
# Perform hyperparameter tuning using GridSearchCV
# param_grid = {
    # You can try different Naive Bayes variations and hyperparameters here
#     'classifier__var_smoothing': [1e-9, 1e-8, 1e-7],
# }
# Define a grid of hyperparameter values to search over
param_grid = {
    'classifier__var_smoothing': np.logspace(-9, 0, 10)  # Adjust the range of values as needed -9 0 10
}
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

In [39]:
import pandas as pd

# Load the sample submission CSV file
submission_df = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')

# Extract the target labels for the first 100k rows (assuming your test data has 100k rows)
y_true = submission_df['target'][:100000]

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming you have a trained SVM model named 'clf'

# Evaluate the model's performance on the test data
accuracy = accuracy_score(y_true, y_pred)
report = classification_report(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
pca_df.head()

In [ ]:
pca_df.shape

In [ ]:
# import numpy as np
# from sklearn.cluster import KMeans

# # Assuming X_train_pca is the PCA-transformed training data
# # n_clusters is the number of clusters you want to create
# n_clusters = 3  # You can adjust this number based on your problem

# # Create a K-Means clustering model
# kmeans = KMeans(n_clusters=n_clusters, random_state=0)

# # Fit the model to your PCA-transformed training data
# kmeans.fit(X_train_pca)

# # Get the cluster labels for each data point
# cluster_labels = kmeans.labels_

# You can now use cluster_labels for classification or analysis


In [ ]:
# from sklearn.model_selection import train_test_split

# # Split the PCA-transformed data into training and test sets
# X_train_pca, X_test_pca, = train_test_split(X_projected, test_size=0.3333333, random_state=42)

# # Convert NumPy arrays to Pandas DataFrames
# X_train_pca_df = pd.DataFrame(data=X_train_pca, columns=[f'PC{i}' for i in range(1, 34)])
# X_test_pca_df = pd.DataFrame(data=X_test_pca, columns=[f'PC{i}' for i in range(1, 34)])



In [ ]:
X_train_pca.shape

In [ ]:
# X_test_pca_df

In [ ]:
X_test_pca.shape

In [ ]:
pip install --upgrade threadpoolctl


In [ ]:
print("Columns in pca_df:", pca_df.columns)
print("Columns in train:", train.columns)


In [ ]:
print("Index of pca_df:", pca_df.index)
print("Index of train:", train.index)


In [ ]:
print("Columns in train:", train.columns)



In [99]:
#K-means algorithm
    
K-means algorithm
    
import pandas as pd
import numpy as np
from cuml.cluster import KMeans as cuKMeans
from cudf import DataFrame as cudf_DatamFrame

# Assuming df is your original DataFrame with PC1 to PC33 and the target column
# The target column should be named 'target'

# Extract numerical features for clustering
features = pca_df.columns[:-1]  # Exclude the target column
data_for_clustering = pca_df[features]

# Define the number of clusters
num_clusters = 10000

# Initialize the KMeans model on GPU
kmeans = cuKMeans(n_clusters=num_clusters, random_state=0, n_init=1)

# Fit KMeans model
cluster_assignments_gpu = kmeans.fit_predict(data_for_clustering)

# Transfer cuML Series to cuDF DataFrame
cluster_assignments_cudf = cudf.Series(cluster_assignments_gpu)

# Convert cuDF Series to a NumPy array
cluster_assignments_cpu = cluster_assignments_cudf.to_pandas().values

# Add cluster assignments to the original DataFrame
df_with_clusters = pd.concat([pca_df, pd.DataFrame({'cluster': cluster_assignments_cpu})], axis=1)

# Initialize a new DataFrame for the reduced dataset
reduced_data = pd.DataFrame(columns=pca_df.columns)

# Randomly select 10,000 rows from the entire DataFrame with replacement
selected_rows = df_with_clusters.sample(n=10000, replace=True, random_state=0)

# Concatenate the selected rows to the reduced_data DataFrame
reduced_data = pd.concat([reduced_data, selected_rows], ignore_index=True)

  # Extract class labels from the "cluster" column
class_labels = reduced_data["cluster"].values

# Drop the "cluster" column from the reduced dataset
reduced_data = reduced_data.drop(columns=["cluster"])

# Save the reduced data
reduced_data.to_csv("reduced_data.csv", index=False)

print(f"Reduced data size: {len(reduced_data)}")

SyntaxError: invalid syntax (1862913539.py, line 3)

In [ ]:
print(reduced_df.columns)


In [29]:
# Reducing rows using K-means Clustering
from sklearn.cluster import KMeans

# Extracting numeric columns from pca_df
numeric_columns = pca_df.select_dtypes(include=[np.number])
num_clusters = 10000

# Initialize KMeans with 'num_clusters' clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=0, n_init=1)  #random_state is a parameter that controls the random number generator used by the algorithm.
                                                                   #n_init controls the number of times the algorithm will be run with different centroid initializations, and in your code, it's set to 1, meaning only one run.
cluster_assignments = kmeans.fit_predict(numeric_columns)
pca_df['cluster'] = cluster_assignments
# Assuming 'target_df' contains the target column and shares the same index as 'pca_df'
merged_df = pd.merge(pca_df, target_column, left_index=True, right_index=True)


averaged_records = []

# Loop through each unique cluster
for cluster_id in merged_df['cluster'].unique():
    cluster_records = merged_df[merged_df['cluster'] == cluster_id]
    
    # Counting occurrences of each class in the cluster
    class_counts = cluster_records['target'].value_counts()
    
    #class with the most occurrences (predominant class)
    predominant_class = class_counts.idxmax()
    
    # Select records of the predominant class and calculate their mean
    predominant_records = cluster_records[cluster_records['target'] == predominant_class]
    averaged_record = predominant_records.drop(columns=['cluster', 'target']).mean()
    
    # Assigning the predominant class to the averaged record
    averaged_record['target'] = predominant_class
    
    # Appending the averaged record to the list
    averaged_records.append(averaged_record)

# Create the 'reduced_df' DataFrame from the list of averaged records
reduced_df = pd.DataFrame(averaged_records)

# 'reduced_df'contains the reduced and averaged records with the predominant class


/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1033: ConvergenceWarning: Number of distinct clusters (4386) found smaller than n_clusters (10000). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_


KeyError: 'target'

In [ ]:
reduced_df.shape

In [ ]:
reduced_df.head()

In [ ]:
# Assuming your target column is named 'target' and you want to use all 33 feature columns
X_train = reduced_df.drop(columns=['target'])  # Extract all feature columns except 'target'
y_train = reduced_df['target']  # Extract the 'target' column as labels


In [ ]:
from sklearn.svm import SVC

# Assuming you have your training data X_train and y_train
# clf = SVC(kernel='poly')  # Example with a linear kernel
# clf.fit(X_train, y_train)

In [ ]:
# Make predictions on the test data
y_pred = clf.predict(X_test_pca)


In [ ]:
import pandas as pd

# Load the sample submission CSV file
submission_df = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')

# Extract the target labels for the first 100k rows (assuming your test data has 100k rows)
y_true = submission_df['target'][:100000]


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming you have a trained SVM model named 'clf'

# Evaluate the model's performance on the test data
accuracy = accuracy_score(y_true, y_pred)
report = classification_report(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)



In [ ]:
# y=reduced_df['target']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
#Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=1000,max_depth=6, random_state=42)
# rf_classifier = RandomForestClassifier(
#     n_estimators=1000,
#     max_depth=6,
#     min_samples_split=5,
#     class_weight="balanced",
#     random_state=42
# )


In [ ]:
# Train the classifier on the training data
rf_classifier.fit(X_train, y_train)

In [ ]:
# Make predictions on the test data
y_pred = rf_classifier.predict(X_test_pca)


In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")



In [ ]:
# You can also print a detailed classification report
report = classification_report(y_true, y_pred )
print("Classification Report:\n", report)

In [ ]:
# print("Accuracy:", accuracy)

In [ ]:
# Naive Bayes classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split, GridSearchCV


In [ ]:
# Create a pipeline with MinMax scaling and Naive Bayes classifier
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),  # Scale features to the [0, 1] range
    ('classifier', GaussianNB())  # Use Gaussian Naive Bayes classifier
])

In [ ]:
# Create a pipeline with MinMax scaling and Naive Bayes classifier
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),  # Scale features to the [0, 1] range
    ('classifier', GaussianNB())  # Use Gaussian Naive Bayes classifier
])
# Perform hyperparameter tuning using GridSearchCV
# param_grid = {
    # You can try different Naive Bayes variations and hyperparameters here
#     'classifier__var_smoothing': [1e-9, 1e-8, 1e-7],
# }
# Define a grid of hyperparameter values to search over
param_grid = {
    'classifier__var_smoothing': np.logspace(-9, 0, 10)  # Adjust the range of values as needed -9 0 10
}
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

In [ ]:
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

In [ ]:
# Get the best estimator from the grid search
best_pipeline = grid_search.best_estimator_

In [ ]:
# Train the best pipeline on the training data
best_pipeline.fit(X_train, y_train)

In [ ]:
# Make predictions on the test data
y_pred = best_pipeline.predict(X_test_pca)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_true, y_pred)
print( accuracy)

In [ ]:
# Initialize a Gaussian Naive Bayes classifier
# nb_classifier = GaussianNB()

In [ ]:
# Train the classifier
# nb_classifier.fit(X_train, y_train)


In [ ]:
# Train the pipeline (scaling and classification)
pipeline.fit(X_train, y_train)

In [ ]:
# Make predictions on the test data
y_pred = pipeline.predict(X_test_pca)

In [16]:
target_column_numpy = np.array(target_column)
train=train.drop(columns=['target']) 

NameError: name 'target_column' is not defined

In [14]:
train

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
0,0,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07,Streptococcus_pyogenes
1,1,-9.536743e-07,-9.536743e-06,-0.000043,0.000886,-0.000200,0.000760,-0.000200,-0.000114,-0.000043,...,-0.000086,-0.000043,0.000914,0.000914,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07,Salmonella_enterica
2,2,-9.536743e-07,-1.536743e-06,0.000007,0.000129,0.000268,0.000270,0.000243,0.000125,0.000001,...,0.000084,0.000048,0.000081,0.000106,0.000072,1.046326e-05,0.000008,0.000019,1.046326e-06,Salmonella_enterica
3,3,4.632568e-08,-5.536743e-06,0.000012,0.000245,0.000492,0.000522,0.000396,0.000197,-0.000003,...,0.000151,0.000100,0.000180,0.000202,0.000153,2.146326e-05,0.000015,0.000046,-9.536743e-07,Salmonella_enterica
4,4,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07,Enterococcus_hirae
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199995,-9.536743e-07,4.632568e-07,-0.000003,0.000176,0.000350,0.000290,0.000200,0.000206,-0.000023,...,0.000124,0.000057,0.000104,0.000144,0.000027,4.632568e-07,0.000060,0.000020,-9.536743e-07,Salmonella_enterica
199996,199996,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000086,-0.000043,0.000914,0.000914,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07,Streptococcus_pyogenes
199997,199997,4.632568e-08,1.463257e-06,-0.000005,-0.000031,-0.000019,-0.000037,-0.000037,-0.000015,-0.000005,...,0.000115,0.000131,0.000110,0.000213,0.000094,1.646326e-05,0.000035,0.000021,4.632568e-08,Streptococcus_pneumoniae
199998,199998,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000086,-0.000043,0.001914,-0.000086,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07,Staphylococcus_aureus


In [15]:
from sklearn.model_selection import train_test_split

X_trainnew_1, X_val_1, y_trainnew_1, y_val_1 = train_test_split(train, target_column, test_size=0.3, random_state=42)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


NameError: name 'target_column' is not defined

In [67]:
X_trainnew_1

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
21269,21269,-9.536743e-07,1.046326e-05,7.084656e-06,-0.000004,-0.000030,0.000020,-0.000050,0.000016,-0.000043,...,0.000117,0.000184,0.000137,0.000234,0.000224,0.000137,2.046326e-05,0.000010,0.000030,-9.536743e-07
187660,187660,4.632568e-08,2.463257e-06,4.808466e-05,0.000348,0.000822,0.001119,0.000856,0.000343,0.000032,...,0.000022,0.000035,0.000033,0.000050,0.000067,0.000042,1.246326e-05,0.000008,0.000018,4.632568e-08
774,774,4.632568e-08,3.463257e-06,8.465576e-08,-0.000015,0.000007,-0.000004,-0.000005,0.000003,0.000003,...,0.000018,0.000064,0.000034,0.000066,0.000069,0.000041,1.246326e-05,0.000016,0.000009,4.632568e-08
184577,184577,-9.536743e-07,-9.536743e-06,-4.291534e-05,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000043,-0.000086,0.000957,-0.000086,-0.000086,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07
37127,37127,-9.536743e-07,-9.536743e-06,-4.291534e-05,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000043,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,119879,-9.536743e-07,-9.536743e-06,-4.291534e-05,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000043,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-9.536743e-06,-0.000010,-0.000010,-9.536743e-07
103694,103694,-9.536743e-07,4.632568e-07,-2.915344e-06,0.000036,0.000110,0.000030,0.000010,0.000036,-0.000023,...,0.000077,0.000104,0.000037,0.000134,0.000114,0.000007,4.632568e-07,0.000020,0.000010,-9.536743e-07
131932,131932,2.046326e-06,-5.367432e-07,-1.391534e-05,-0.000056,-0.000067,-0.000104,-0.000085,-0.000048,-0.000015,...,0.000283,0.000648,0.000390,0.000902,0.001207,0.000885,1.334633e-04,0.000203,0.000227,1.046326e-06
146867,146867,-9.536743e-07,-9.536743e-06,-4.291534e-05,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,0.000957,0.000914,-0.000043,0.000914,0.001914,-0.000043,9.904633e-04,0.000990,-0.000010,-9.536743e-07


In [68]:
X_val_1

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
119737,119737,-9.536743e-07,4.632568e-07,-0.000033,0.000016,0.000070,-0.000030,-0.000010,-0.000034,-0.000003,...,0.000087,0.000204,0.000117,0.000254,0.000154,0.000177,0.000020,0.000030,4.046326e-05,-9.536743e-07
72272,72272,-9.536743e-07,1.046326e-05,0.000007,-0.000004,-0.000030,0.000020,-0.000050,0.000016,-0.000043,...,0.000077,0.000174,0.000137,0.000264,0.000324,0.000267,0.000010,0.000020,5.046326e-05,-9.536743e-07
158154,158154,-9.536743e-07,-6.536743e-06,-0.000023,-0.000069,-0.000112,-0.000146,-0.000113,-0.000066,-0.000021,...,0.000166,0.000501,0.000312,0.000505,0.000557,0.000322,0.000064,0.000099,8.646326e-05,1.046326e-06
65426,65426,4.632568e-08,-8.536743e-06,-0.000018,0.000043,0.000030,-0.000007,0.000002,0.000047,-0.000019,...,0.000097,0.000174,0.000099,0.000199,0.000229,0.000168,0.000023,0.000015,4.646326e-05,-9.536743e-07
30074,30074,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,0.009800,0.009886,-0.000043,...,-0.000043,-0.000086,-0.000043,-0.000086,-0.000086,0.009957,-0.000010,-0.000010,-9.536743e-06,-9.536743e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97771,97771,4.632568e-08,1.463257e-06,0.000002,0.000002,0.000033,0.000013,0.000020,0.000018,0.000004,...,0.000023,0.000039,0.000026,0.000041,0.000053,0.000028,0.000003,0.000003,1.046326e-05,4.632568e-08
59813,59813,-9.536743e-07,1.046326e-05,0.000007,-0.000004,-0.000020,0.000020,-0.000050,0.000016,-0.000043,...,0.000157,0.000384,0.000257,0.000294,0.000414,0.000147,0.000020,0.000060,5.046326e-05,-9.536743e-07
103735,103735,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000043,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-9.536743e-06,-9.536743e-07
180226,180226,-9.536743e-07,1.046326e-05,0.000007,0.000056,0.000040,0.000090,0.000040,0.000106,-0.000033,...,0.000017,0.000124,0.000107,0.000114,0.000164,0.000007,0.000010,0.000030,4.632568e-07,-9.536743e-07


In [69]:
import numpy as np

# Convert a list of lists to a NumPy array
X_train = np.array(X_trainnew_1)
# Assuming X_train is your training data
# X_train should be a 2D NumPy array

# Initialize an empty array to store the scaled data
X_train_scaled = np.zeros_like(X_train)


# Iterate through each row of X_train
for i in range(X_train.shape[0]):
    row = X_train[i, :]  # Get the current row
    min_val = np.min(row)  # Calculate the minimum value in the row
    max_val = np.max(row)  # Calculate the maximum value in the row

    # Avoid division by zero
    if max_val != min_val:
        scaled_row = (row - min_val) / (max_val - min_val)
    else:
        scaled_row = np.zeros_like(row)  # If max and min are the same, set all values to 0
         # Debugging: Print scaled_row for each row
#     print(f"Scaled Row {i}: {scaled_row}")

    X_train_scaled[i, :] = scaled_row  # Store the scaled row in the result array

# Now, X_train_scaled contains your normalized training data with each row scaled individually between 0 and 1


In [70]:
print(X_train_scaled)

[[1.00000000e+00 1.48179916e-07 1.48716703e-07 ... 1.48716703e-07
  1.49657039e-07 1.48179916e-07]
 [1.00000000e+00 3.54291744e-08 3.54420537e-08 ... 3.54740264e-08
  3.55273143e-08 3.54291744e-08]
 [1.00000000e+00 1.81219193e-06 1.81660657e-06 ... 1.83340240e-06
  1.82435849e-06 1.81219193e-06]
 ...
 [1.00000000e+00 7.22920778e-08 7.22724990e-08 ... 7.38187500e-08
  7.40006619e-08 7.22844981e-08]
 [1.00000000e+00 9.54842787e-08 9.54258377e-08 ... 1.02234719e-07
  9.54258377e-08 9.54842787e-08]
 [1.00000000e+00 7.32681615e-08 7.32223829e-08 ... 7.40669359e-08
  7.39849405e-08 7.32763611e-08]]


In [71]:
import numpy as np

# Assuming X_train_scaled is your scaled dataset
# Create an empty array for the formatted data
X_train_formatted = np.empty_like(X_train_scaled, dtype=np.float64)

# Iterate through each row and column of X_train_scaled
for i in range(X_train_scaled.shape[0]):
    for j in range(X_train_scaled.shape[1]):
        scaled_value = X_train_scaled[i, j]
        formatted_value = "{:.5f}".format(scaled_value)
        X_train_formatted[i, j] = float(formatted_value)

# Now, X_train_formatted c

In [72]:
print(X_train_formatted)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [81]:
# Convert a list of lists to a NumPy array
X_test = np.array(X_val_1)

In [84]:
X_test.shape


(60000, 287)

In [86]:
X_test = X_test.astype(np.float64)


In [87]:
X_test.shape

(60000, 287)

In [88]:
import numpy as np

# Assuming X_train is your training data (a 2D NumPy array)
# and X_test is your test data (a 2D NumPy array)

# Initialize an empty array for scaled test data
X_test_scaled = np.zeros_like(X_test)

# Iterate through each row of X_test
for i in range(X_test.shape[0]):
    row = X_test[i, :]  # Get the current row
    min_val = np.min(X_train[i, :])  # Calculate the minimum value in the corresponding training data row
    max_val = np.max(X_train[i, :])  # Calculate the maximum value in the corresponding training data row

    # Scale the row to have values between 0 and 1 using training data's min and max
    if max_val != min_val:
        scaled_row = (row - min_val) / (max_val - min_val)
    else:
        scaled_row = np.zeros_like(row)  # If max and min are the same, set all values to 0

    X_test_scaled[i, :] = scaled_row  # Store the scaled row in the result array

# Now, X_test_scaled contains your scaled test data with each row scaled using training data's min and max


In [89]:
X_test_scaled

array([[5.62964810e+00, 1.48179916e-07, 1.48246535e-07, ...,
        1.49657039e-07, 1.50127207e-07, 1.48179916e-07],
       [3.85122051e-01, 3.54238456e-08, 3.54846840e-08, ...,
        3.55379718e-08, 3.56978354e-08, 3.54238456e-08],
       [2.04332965e+02, 1.81089995e-06, 1.80368669e-06, ...,
        1.94063735e-06, 1.92384151e-06, 1.81348392e-06],
       ...,
       [5.33778282e-01, 5.65585156e-08, 5.65143506e-08, ...,
        5.65143506e-08, 5.65143506e-08, 5.65585156e-08],
       [2.09392236e+00, 8.86667800e-08, 8.87994255e-08, ...,
        8.90317918e-08, 8.86832424e-08, 8.86667800e-08],
       [1.25031673e+00, 2.51674396e-07, 2.51584509e-07, ...,
        2.51584509e-07, 2.51584509e-07, 2.51674396e-07]])

In [90]:
X_train_formatted.shape

(140000, 287)

In [91]:
import numpy as np
from sklearn.decomposition import PCA

# Assuming X_train_scaled is your scaled training data
# You can use X_train_formatted if you've converted it to human-readable format X_train_formatted

# Initialize the PCA model with the desired number of components
n_components = 33  # Adjust this value as needed
pca = PCA(n_components=n_components)

# Fit the PCA model to your scaled data
pca.fit(X_train_formatted)

# Transform the data to its principal components
X_train_pca = pca.transform(X_train_formatted)
# Create a DataFrame for the PCA results using common columns
pca_df = pd.DataFrame(data=X_train_pca, columns=[f'PC{i}' for i in range(1, 34)])
# Printing the PCA results
print(pca_df)

# Now, X_train_pca contains your training data reduced to the specified number of principal components
# You can use X_train

             PC1           PC2           PC3           PC4           PC5  \
0      -0.000008  9.523237e-07 -2.790328e-08 -9.923074e-08 -1.745966e-07   
1      -0.000008  9.523237e-07 -2.790328e-08 -9.923074e-08 -1.745966e-07   
2      -0.000008  9.523237e-07 -2.790328e-08 -9.923074e-08 -1.745966e-07   
3      -0.000008  9.523237e-07 -2.790328e-08 -9.923074e-08 -1.745966e-07   
4      -0.000008  9.523237e-07 -2.790328e-08 -9.923074e-08 -1.745966e-07   
...          ...           ...           ...           ...           ...   
139995 -0.000008  9.523237e-07 -2.790328e-08 -9.923074e-08 -1.745966e-07   
139996 -0.000008  9.523237e-07 -2.790328e-08 -9.923074e-08 -1.745966e-07   
139997 -0.000008  9.523237e-07 -2.790328e-08 -9.923074e-08 -1.745966e-07   
139998 -0.000008  9.523237e-07 -2.790328e-08 -9.923074e-08 -1.745966e-07   
139999 -0.000008  9.523237e-07 -2.790328e-08 -9.923074e-08 -1.745966e-07   

                 PC6           PC7           PC8           PC9          PC10  \
0      

In [92]:
                   # pca of test data
import numpy as np
from sklearn.decomposition import PCA

# Assuming X_test_scaled is your scaled training data
# You can use X_test_formatted if you've converted it to human-readable format X_test_formatted

# Initialize the PCA model with the desired number of components
n_components = 33  # Adjust this value as needed
pca = PCA(n_components=n_components)

# Fit the PCA model to your scaled data
pca.fit(X_test_scaled)

# Transform the data to its principal components
pca_df_test = pca.transform(X_test_scaled)
# Create a DataFrame for the PCA results using common columns
pca_df_test = pd.DataFrame(pca_df_test, columns=[f'PC{i}' for i in range(1, 34)])
# Printing the PCA results
print(pca_df_test)

              PC1           PC2           PC3           PC4           PC5  \
0       -2.855412 -8.429165e-06  3.276845e-07  9.681002e-07  3.831353e-08   
1       -8.099938 -7.474848e-06  6.038454e-07  5.913135e-07  1.584211e-07   
2      195.847904 -8.473639e-05  2.141763e-05 -2.625276e-05 -1.511080e-05   
3       -8.130596 -7.241614e-06  5.115289e-07  6.730889e-07  1.729805e-07   
4       -7.675030 -1.246280e-07 -5.208269e-07  6.931755e-07  2.871604e-07   
...           ...           ...           ...           ...           ...   
59995   -7.686982 -5.037773e-06  7.795984e-08  6.227715e-07  1.851122e-07   
59996   -8.156311 -6.937412e-06  4.952560e-07  5.911518e-07  1.666011e-07   
59997   -7.951282 -7.169528e-06  7.179156e-07  5.608076e-07  2.468376e-07   
59998   -6.391138 -7.522186e-06  4.275331e-07  7.434147e-07  1.727079e-07   
59999   -7.234744 -4.293104e-06  2.393909e-07  4.975833e-07  7.657401e-07   

                PC6           PC7           PC8           PC9          PC10

In [ ]:
# Add the target_column to pca_df
pca_df['target'] = y_trainnew_1

In [ ]:
print(pca_df)

In [ ]:
                ##       K-MEAN
import pandas as pd
import cupy as cp  # Assuming you have CuPy installed
from cuml.cluster import KMeans as cuKMeans  # Assuming you have cuML installed

# Assuming you have your data in a DataFrame called df
# Features used for clustering (adjust as needed)
# features_for_clustering = ['feature1', 'feature2', 'feature3']

# Number of clusters (adjust as needed)
num_clusters = 1000

# Assuming you have a DataFrame called pca_df
# Fit K-means model using cuML
kmeans = cuKMeans(n_clusters=num_clusters, random_state=0, n_init=100)
pca_df['cluster_id'] = kmeans.fit_predict(cp.asarray(pca_df.drop('target', axis=1))).get()

# Initialize a new DataFrame for the optimized data
new_df = pd.DataFrame()

# Loop through each cluster
for cluster_id in range(num_clusters):
    each_cluster = pca_df[pca_df['cluster_id'].values == cluster_id]
    
    # Check if the cluster is not empty
    if not each_cluster.empty:
        # Calculate the predominant class in the cluster
        predominant_class = each_cluster['target'].value_counts().idxmax()
        
        # Filter records of the predominant class
        each_cluster_filtered = each_cluster[each_cluster['target'] == predominant_class]
        
        # Calculate the average of the records in the cluster
        averaged_record = each_cluster_filtered.mean(numeric_only=True)
        
        # Assign the predominant class to the averaged record
        averaged_record['target'] = predominant_class
        
        # Append the averaged record to the new DataFrame
        new_df = pd.concat([new_df, averaged_record.to_frame().transpose()], ignore_index=True)

# Display the new DataFrame
# print(new_df)


In [101]:
new_df.shape

(100, 35)

In [ ]:
# C4.5 Decision Tree classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import  GridSearchCV
from sklearn.metrics import accuracy_score

In [ ]:
# Create a C4.5 Decision Tree classifier
# c45_classifier = DecisionTreeClassifier(criterion='entropy',splitter='best')

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Create a DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier()

In [ ]:
# Train the classifier on the training data
dt_classifier.fit(X_train, y_train)


In [ ]:
# Make predictions on the test data
y_pred = dt_classifier.predict(X_test_pca)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")

In [ ]:
# accuracy = accuracy_score(y_true, y_pred)
# print("Accuracy:", accuracy)

In [ ]:
# Using Gini impurity
# c45_classifier_gini = DecisionTreeClassifier(criterion='gini')


In [ ]:
# c45_classifier_gini.fit(X_train, y_train)

In [ ]:
# Step 4: Make predictions on the test data
# y_pred = c45_classifier_gini.predict(X_test_pca)

In [ ]:
# Step 5: Evaluate the model
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)